In [1]:
!dir "examples/text2SQL-solutions/"

 Volume in drive C is OS
 Volume Serial Number is 4A0B-27B2

 Directory of c:\Users\Pratyush\Desktop\text-to-json\examples\text2SQL-solutions

05-05-2025  20:26    <DIR>          .
05-05-2025  20:26    <DIR>          ..
05-05-2025  18:49         2,161,792 chase-sql.pdf
05-05-2025  20:21         1,493,944 DBCopilot.pdf
05-05-2025  20:26                 0 input.txt
05-05-2025  20:23           918,547 RatSQL.pdf
05-05-2025  20:21           341,083 RyanSQL.pdf
05-05-2025  20:26                 0 schema.json
05-05-2025  20:18           454,000 seq2SQL.pdf
05-05-2025  20:19           443,530 sqlNet.pdf
05-05-2025  20:20           728,921 SyntaxSQLNet.pdf
05-05-2025  20:23         2,970,020 t2s-intermediate-rep.pdf
05-05-2025  20:25         1,095,621 t2s-LLMs.pdf
05-05-2025  20:20           614,703 UniSAr.pdf
05-05-2025  18:48         4,588,484 xiyang-sql.pdf
              13 File(s)     15,810,645 bytes
               2 Dir(s)  380,987,977,728 bytes free


In [4]:
import pymupdf
import os


BASE_PATH = "examples/text2SQL-solutions/"
pdf_paths = [os.path.join(BASE_PATH, x) for x in os.listdir(BASE_PATH) if x.endswith('pdf')]
pdf_paths

['examples/text2SQL-solutions/chase-sql.pdf',
 'examples/text2SQL-solutions/DBCopilot.pdf',
 'examples/text2SQL-solutions/RatSQL.pdf',
 'examples/text2SQL-solutions/RyanSQL.pdf',
 'examples/text2SQL-solutions/seq2SQL.pdf',
 'examples/text2SQL-solutions/sqlNet.pdf',
 'examples/text2SQL-solutions/SyntaxSQLNet.pdf',
 'examples/text2SQL-solutions/t2s-intermediate-rep.pdf',
 'examples/text2SQL-solutions/t2s-LLMs.pdf',
 'examples/text2SQL-solutions/UniSAr.pdf',
 'examples/text2SQL-solutions/xiyang-sql.pdf']

In [15]:
APIKEY = "Bearer premKey_zB4JsYhqRtxz2qiPSukLt0IEBsZHJTkQ9AUh"

In [ ]:
import requests

url = "https://sid.premai.io/api/chat/completions"
headers = {
    "Authorization": APIKEY,
    "Content-Type": "application/json"
}
payload = {
    "model": "gpt-4o",
    "messages": [{"role": "user", "content": "hi"}],
    "stream": False,
    "temperature": 0.5
}

response = requests.post(url, headers=headers, json=payload, stream=False)


In [14]:
response.json()

{'id': 'chatcmpl-BTs2OWZcc37VqkCCipiHFtXTFjA3z',
 'object': 'chat.completion',
 'created': 1746458932,
 'model': 'gpt-4o-2024-08-06',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hello! How can I assist you today?',
    'refusal': None,
    'annotations': []},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 8,
  'completion_tokens': 10,
  'total_tokens': 18,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_f5bdcc3276'}

In [22]:
import pymupdf
import concurrent.futures
import json
from tqdm import tqdm
from threading import Lock

N = 100  # Number of lines per chunk

def read_and_chunk(file_path, lines_per_chunk=N):
    doc = pymupdf.open(file_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    doc.close()

    # Split text into lines and chunk them
    lines = full_text.splitlines()
    chunks = [
        "".join(lines[i:i + lines_per_chunk])
        for i in range(0, len(lines), lines_per_chunk)
    ]
    return chunks

def make_prompt(text_chunk):
    return f"""
You are an assistant that extracts headings and subheadings from academic or technical papers.

The following text is a chunk from a paper. Your task is:
1. Identify and extract **section headings, subheadings, and other headings** from the text.
2. Prefer numbered headings if present (e.g., "1. Introduction", "2.1 Previous Work").
3. Only return **phrases that clearly represent headings/subheadings**.

ONLY Return the output as a JSON list of phrases in the order they appear.

Text:
\"\"\"
{text_chunk}
\"\"\"
"""

def call_openapi_gpt(text_chunk):
    url = "https://sid.premai.io/api/chat/completions"
    headers = {
        "Authorization": APIKEY,
        "Content-Type": "application/json"
    }

    prompt = make_prompt(text_chunk)

    payload = {
        "model": "gpt-4o",
        "messages": [{"role": "user", "content": prompt}],
        "stream": False,
        "temperature": 0.3
    }

    response = requests.post(url, headers=headers, json=payload)
    try:
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Error: {e}")
        return None
    

def extract_headings_from_document(file_path):
    chunks = read_and_chunk(file_path)
    all_headings = [None] * len(chunks)
    lock = Lock()

    with tqdm(total=len(chunks), desc="Processing chunks") as pbar:
        def process_chunk(i, chunk):
            result = call_openapi_gpt(chunk)
            headings = []
            if result:
                try:
                    cleaned = result.replace('```json', "").replace("```", "").strip()
                    headings = json.loads(cleaned)
                except Exception as parse_error:
                    print(f"Parse error in chunk {i+1}: {parse_error}")
            with lock:
                all_headings[i] = {
                    "chunk_id": i + 1,
                    "headings": headings
                }
                pbar.update(1)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [
                executor.submit(process_chunk, i, chunk)
                for i, chunk in enumerate(chunks)
            ]
            concurrent.futures.wait(futures)

    return all_headings

# Example usage
if __name__ == "__main__":
    FILE_PATH = r"C:\Users\Pratyush\Desktop\text-to-json\examples\text2SQL-solutions\chase-sql.pdf"
    headings = extract_headings_from_document(FILE_PATH)
    print("\nExtracted Headings & Subheadings:")
    # for h in headings:
    #     print("-", h)

Processing chunks: 100%|██████████| 15/15 [00:05<00:00,  2.95it/s]


Extracted Headings & Subheadings:


In [1]:
from t2j.prem_sdk import PremSDK  
from t2j.prompts import Prompts
from t2j.chunker import DocumentChunker
from t2j.utils import *

N = 100
promptsClass = Prompts()
model = PremSDK()
chunker = DocumentChunker(prompts=promptsClass, model=model)

FILE_PATH = r"C:/Users/Pratyush/Desktop/text-to-json/examples/text2SQL-solutions/SyntaxSQLNet.pdf"
headings = chunker.smart_chunk(FILE_PATH)
headings


100%|██████████| 56/56 [00:00<?, ?it/s]


[{'heading': 'abstract',
  'content:': 'Most existing studies in text-to-SQL tasks do\nnot require generating complex SQL queries\nwith multiple clauses or sub-queries, and gen-\neralizing to new, unseen databases.\nIn this\npaper we propose SyntaxSQLNet, a syntax\ntree network to address the complex and cross-\ndomain text-to-SQL generation task.\nSyn-\ntaxSQLNet employs a SQL specic syn-\ntax tree-based decoder with SQL generation\npath history and table-aware column atten-\ntion encoders.\nWe evaluate SyntaxSQLNet\non the Spider text-to-SQL task, which con-\ntains databases with multiple tables and com-\nplex SQL queries with multiple SQL clauses\nand nested queries. We use a database split\nsetting where databases in the test set are un-\nseen during training.\nExperimental results\nshow that SyntaxSQLNet can handle a signi-\ncantly greater number of complex SQL exam-\nples than prior work, outperforming the pre-\nvious state-of-the-art model by 7.3% in ex-\nact matching accuracy.\

In [ ]:
[{'heading': 'head1', 'content': '...'}, {'heading': 'subhead1', 'content': '...'}, {'heading': 'subhead2', 'content': '...'}, {'heading': 'head1', 'content': '...'}'subhead1', 'subhead2', 'head2', 'subhead3', ...]

In [1]:
from t2j.prem_sdk import PremSDK  
from t2j.prompts import Prompts
from t2j.chunker import DocumentChunker
from t2j.utils import *

import json

N = 100
promptsClass = Prompts()
model = PremSDK()
chunker = DocumentChunker(prompts=promptsClass, model=model)

FILE_PATH = r"C:\Users\Pratyush\Desktop\text-to-json\examples\text2SQL-solutions\chase-sql.pdf"
output = chunker.smart_chunk(FILE_PATH)
print(json.dumps(output, indent=4))

100%|██████████| 28/28 [00:00<00:00, 28008.71it/s]


[{'heading': 'abstract', 'content': 'In tackling the challenges of large language model (LLM) performance for Text-to-SQL tasks, we introduce CHASE-SQL, a new framework that employs innovative strategies, using test-time compute in multi-agent modeling to improve candidate generation and selection. CHASE-SQL leverages LLMs intrinsic knowledge to generate diverse and high-quality SQL candidates using different LLM generators with: (1) a divide-and-conquer method that decomposes complex queries into manageable sub-queries in a single LLM call; (2) chain-of-thought reasoning based on query execution plans, reflecting the steps a database engine takes during execution; and (3) a unique instance-aware synthetic example generation technique, which offers specific few-shot demonstrations tailored to test questions. To identify the best candidate, a selection agent is employed to rank the candidates through pairwise comparisons with a fine-tuned binary-candidates selection LLM. This selection 

In [6]:
headings[0]

{'heading': 'abstract',
 'content:': 'In tackling the challenges of large language model (LLM) performance for Text-to-SQL tasks, we introduce CHASE-SQL, a new framework that employs innovative strategies, using test-time compute in multi-agent modeling to improve candidate generation and selection. CHASE-SQL leverages LLMs intrinsic knowledge to generate diverse and high-quality SQL candidates using different LLM generators with: (1) a divide-and-conquer method that decomposes complex queries into manageable sub-queries in a single LLM call; (2) chain-of-thought reasoning based on query execution plans, reflecting the steps a database engine takes during execution; and (3) a unique instance-aware synthetic example generation technique, which offers specific few-shot demonstrations tailored to test questions. To identify the best candidate, a selection agent is employed to rank the candidates through pairwise comparisons with a fine-tuned binary-candidates selection LLM. This selection

In [2]:
lines = extract_text(FILE_PATH)
chunks = chunk(lines)

In [4]:
print(chunks[7])

Q/CS
⊤+ Wval
2 Hval
HS/CS
⊤+ Wval
3 HCS⊤
Root/Terminal Module
To predict nested sub-
queries, we add a module to predict if there is a
new “ROOT” after an operator, which allows the
model to decode queries recursively. For each pre-
dicted column from the COL module that is in
the WHERE clause, we ﬁrst call OP module, and
then predict whether the next decoding step is a
“ROOT” node or a value terminal node by
PRT = P

W1H⊤
Q/CS + W2H⊤
HS/CS + W3H⊤
CS

AND/OR Module
For each condition column
predicted from the COL module with number big-
ger than 1, we predict from {AND, OR} by
PAO = P

W1H⊤
Q + W2H⊤
HS

DESC/ASC/LIMIT Module
In this module,
for each predicted column from the COL module
that is in the ORDER BY clause, we predict
from {DESC, ASC, DESC LIMIT, ASC LIMIT} by
PDAL = P

W1H⊤
Q/CS + W2H⊤
HS/CS + W3H⊤
CS

HAVING Module
In the HAVING module, for
each predicted column from the COL module that
is in the GROUP BY clause, we predict whether it
is in the HAVING clause by
PHAVI

In [ ]:
final_headings = []
from tqdm import tqdm

for i, head in tqdm(enumerate(headings), total=len(headings)):
    text_chunk = chunks[head['chunk_id']]
    text_chunk = text_chunk.replace("\n", " ")
    if i+1 != len(headings):
        if headings[i+1]['chunk_id'] != head['chunk_id']:
            text_chunk += (" " + chunks[headings[i+1]['chunk_id']])
        text_chunk = text_chunk.replace("\n", " ")
        content = text_chunk.split(head['heading'], maxsplit=1)[1].split(headings[i+1]['heading'], maxsplit=1)[0]
        final_headings.append({
            'heading': head['heading'],
            'chunk_id': head['chunk_id'],
            'content': content
        })
    else:
        content = text_chunk.split(head['heading'], maxsplit=1)[1]
        final_headings.append({
            'heading': head['heading'],
            'chunk_id': head['chunk_id'],
            'content': content
        })

100%|██████████| 26/26 [00:00<?, ?it/s]


In [1]:
final_headings = [{'heading': '1 Introduction',
  'chunk_id': 0,
  'content': ' Text-to-SQL, as a bridge between human language and machine-readable structured query languages, is crucial for many use cases, converting natural language questions into executable SQL commands (Androutsopoulos et al., 1995; Li & Jagadish, 2014; Li et al., 2024c; Yu et al., 2018; ?). By enabling users to interact with complex database systems without requiring SQL proficiency, Text-to-SQL empowers users to extract valuable insights, perform streamlined data exploration, make informed decisions, generate data-driven reports and mine better features for machine learning (Chen et al., 2023; Pérez-Mercado et al., 2023; Pourreza & Rafiei, 2024a; Pourreza et al., 2024; Sun et al., 2023; Wang et al., 2019; Xie et al., 2023). Furthermore, Text-to-SQL systems play a pivotal role in automating data analytics with complex reasoning and powering conversational agents, expanding their applications beyond traditional data retrieval (Sun et al., 2023; Xie et al., 2023). As data continues to grow exponentially, the ability to query databases efficiently without extensive SQL knowledge becomes increasingly vital for a broad range of applications. Text-to-SQL can be considered a specialized form of code generation, with the contextual information potentially including the database schema, its metadata and along with the values. In the broader code generation domain, utilizing LLMs to generate a wide range of diverse candidates and select the best one has proven to be effective (Chen et al., 2021; Li et al., 2022; Ni et al., 2023). However, it is non-obvious what 1 arXiv:2410.01943v1  [cs.LG]  2 Oct 2024 leads to most effective candidate proposal and winner selector mechanisms. A straightforward yet effective approach involves generating candidates using zero-/few-shot or open-ended prompting, followed by selecting the best options utilizing self-consistency (Wang et al., 2022), which entails clustering candidates based on their execution outputs. This approach has demonstrated promising results in several studies (Lee et al., 2024; Maamari et al., 2024; Talaei et al., 2024; Wang et al., 2023). However, a single prompt design might not fully unleash the extensive Text-to-SQL knowledge of LLMs, and self-consistency methods might not be always effective. In fact, as illustrated in Table 1, the most consistent answers would not always be the correct ones, with an upper-bound performance 14% higher than that achieved through self-consistency. This substantial gap highlights the potential for significant improvement by implementing more effective selection methods to identify the best answer from the pool of candidate queries. Table 1: Evaluating single-query gen- eration vs. ensemble methods of self- consistency and the upper bound that can be achieved for Text-to-SQL with Gemini 1.5 Pro on the BIRD dev set. EX stands for execution accuracy. Method EX (%) Single query 63.01 Self-consistency 68.84 (+ 5.84) Upper-bound 82.79 (+ 19.78) Building on the challenges outlined in the previous section, we pro- pose novel approaches to improve LLM performance for Text-to-SQL by leveraging judiciously-designed test-time computations in an agentic framework. As indicated by the upper bound in Table 1, utilizing LLMs’ intrinsic knowledge offers significant potential for improvement. We propose methods that generate a diverse set of high-quality can- didate responses and apply a selection mechanism to identify the best answer. Achieving both high-quality and diverse candidate responses is critical for the success of scoring-based selection methods. Low diversity limits improvement potential and reduces the difference be- tween self-consistency and scoring-based approaches. While techniques like increasing temperature or reordering prompt contents can boost diversity, they often compromise the quality of the candidates. To address this, we introduce effective candidate generators designed to enhance diversity while maintaining high-quality outputs. Specifically, we propose three distinct candidate generation approaches, each capable of producing high-quality responses. The first is inspired by the divide-and-conquer algorithm, which breaks down complex problems into smaller, manageable parts to handle difficult queries. The second employs a query execution-plan-based chain-of-thought strategy, where the reasoning process mirrors the steps a database engine takes during query execution. Lastly, we introduce a novel online synthetic example generation method, which helps the model better understand the underlying data schema of the test database. These methods, when used independently, can produce highly-accurate SQL outputs. To effectively select the best answer among candidates, we introduce a selection agent, trained with a classification objective, that assigns scores based on pairwise comparisons between candidate queries. With this agent, we construct a comparison matrix for all candidates and select the final response based on the highest cumulative score. By combining these candidate generation methods with the proposed scoring model, we create an ensemble approach that leverages the strengths of each strategy to significantly improve overall performance. We present comprehensive evaluations on the efficacy of proposed methodologies of CHASE-SQL. Our innovative candidate generation approaches demonstrate superior performance compared to traditional generic CoT prompts, illustrating their capability in guiding LLMs through the decomposition of complex problems into manageable intermediate steps. Furthermore, the proposed selection agent significantly outperforms conventional consistency-based methods, contributing to the state-of-the-art results. Specifically, CHASE- SQL reaches an execution accuracy of 73.01% and 73.0% on the development set and test set of the challenging BIRD Text-to-SQL dataset which outperforms all of the published and undisclosed methods on this benchmark, by a large margin. '},
 {'heading': '2 Related Work',
  'chunk_id': 1,
  'content': ' Early Text-to-SQL methods predominantly utilized sequence-to-sequence architectures, encoding user queries and database schemas using models such as Graph Neural Networks (GNNs), Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, and pre-trained transformer encoders (Cai et al., 2021; Cao et al., 2021; Hwang et al., 2019). On the decoding side, these systems employed either slot-filling or auto-regressive modelling approaches to construct the final SQL queries from the encoded inputs (Choi et al., 2021; Wang et al., 2019). Additionally, tabular language models like TaBERT (Yin et al., 2020), 2 TaPas (Herzig et al., 2020), and Grappa (Yu et al., 2020) have been developed to encode both tables and textual data effectively. However, the landscape has evolved with the widespread use of LLMs, which have largely replaced earlier methods with their superior performance (Katsogiannis-Meimarakis & Koutrika, 2023; Quamar et al., 2022). Initially, efforts concentrated on optimizing prompt designs for these LLMs (Dong et al., 2023; Gao et al., 2023; Pourreza & Rafiei, 2024a). Subsequent advancements have introduced more complex methodologies, including schema linking (Li et al., 2024b; Pourreza & Rafiei, 2024a,b; Talaei et al., 2024), self-correction or self-debugging (Chen et al., 2023; Talaei et al., 2024; Wang et al., 2023), and self-consistency techniques (Lee et al., 2024; Maamari et al., 2024; Sun et al., 2023; Talaei et al., 2024), further enhancing the performance by proposing complex LLM-based pipelines. '},
 {'heading': '3 Methods', 'chunk_id': 1, 'content': ' '},
 {'heading': '3.1 Overall Framework',
  'chunk_id': 1,
  'content': ' This section outlines the proposed CHASE-SQL framework, which consists of four primary components: 1) Value retrieval, 2) Candidate generator, 3) Query fixer, and 4) Selection agent. As illustrated in Fig. 1. The proposed framework begins by retrieving relevant database values. Subsequently, all contextual information, including retrieved values, database metadata, and schema, is provided to an LLM to generate candidate queries. These candidate queries then undergo a fixing loop, and finally, all candidates are compared in a pairwise way using the trained selection agent to pick the correct answer. The following sections delve into the details of each component. Figure 1: Overview of the proposed CHASE-SQL framework for Text-to-SQL, with value retrieval and using a selection agent for improve picking of the answers among the generated candidates along with a fixer to provide feedback for refinement of the outputs. '},
 {'heading': '3.2 Value Retrieval',
  'chunk_id': 1,
  'content': ' Databases might contain very high number of rows, with often only a few being relevant to a query. Retrieving relevant values is crucial as they can be used in various SQL clauses like ‘WHERE’ and ‘HAVING’. Similar to the approach in (Talaei et al., 2024), we begin by extracting keywords from the given question using an LLM prompted with few-shot examples. For each keyword, we employ locality-sensitive hashing (LSH) (Datar et al., 2004) to retrieve the most syntactically-similar words, and re-rank them based on embedding-based similarity and edit distance. This approach is robust to typos in the question and considers keyword semantics during retrieval. '},
 {'heading': '3.3 Multi-path Candidate Generation',
  'chunk_id': 1,
  'content': ' As shown in Table 1, relying solely on consistency among responses can lead to sub-optimal performance. Therefore, we prioritize diversity in generation of multiple response candidates to increase the likelihood of generating at least one correct answer. Among the diverse responses generated by the candidate generators, we select one as the final response using a selection agent that compares candidates pairwise. To generate 3 diverse responses, we increase the next token sampling temperature, and also shuffle the order of columns and tables in the prompt. Chain-of-Thought (CoT) prompting (Wei et al., 2022) has been proposed to enhance LLMs’ reasoning abilities by conditioning their final responses on a step-by-step chain of reasoning. Most CoT prompting approaches rely on few-shot examples in the prompt to guide LLMs on thinking step-by-step, following the format M = (qi, ri, si), where qi is the example question, ri is the reasoning path, and si is the ground truth SQL query for qi. We employ two distinct reasoning methods and an online synthetic example generation approach. As shown in Fig. 3a, different generators can yield different outputs, indicating their effectiveness for specific questions and databases. '},
 {'heading': 'Divide and Conquer CoT',
  'chunk_id': 1,
  'content': ': Divide-and-conquer perspective brings breaking down complex problems into smaller sub-problems, solving each individually, and then combining the solutions to obtain the final answer. Along these lines, we propose a CoT prompting approach that first decomposes the given question into smaller sub-problems using pseudo-SQL queries. In the ’conquer’ step, the solutions to these sub-problems are aggregated to construct the final answer. Finally, an optimization step is applied to the constructed query to remove redundant clauses and conditions. This approach is particularly powerful handling complex scenarios that involve nested queries, e.g. intricate WHERE or HAVING conditions, and queries requiring advanced mathematical operations. In Appendix Fig. 17, we exemplify a question and its corresponding SQL query that was successfully solved using this generator, a scenario the other methods considered in this paper could not address due to the query’s complex conditions and SQL clauses. For a more detailed view of the divide-and-conquer prompt, please see Appendix Fig. 16. Additionally, Alg. 1 outlines the step-by-step process of this strategy to generate the final SQL output using a single LLM call. Algorithm 1 Divide and Conquer Chain-of-Thought (CoT) Strategy for Text-to-SQL. Input: Set of human-annotated few-shot examples M, user question Qu, target database D associated with the question, and a large language model (LLM) θ. Divide: 1: Sq ←θ(M, D, Qu) // Decompose the original question Qu into a set of sub-questions Sq 2: Ssql ←∅// Initialize an empty set Ssql to store partial SQL queries for each sub-question Conquer: 3: for each sub-question qi in Sq do 4: // Generate a partial SQL query for each sub-question qi 5: Ssql ←Ssql ∪{θ(M, D, Qu, q1, ..., qi, sql1, ..., sqli−1)} 6: end for Assemble: 7: Sf ←θ(M, D, Qu, Sq, Ssql) // Assemble the final SQL query Sf from all sub-queries in Ssql 8: return Sf '},
 {'heading': 'Query Plan CoT',
  'chunk_id': 1,
  'content': ': A query (execution) plan is a sequence of steps that the database engine follows to access or modify the data described by a SQL command. When a SQL query is executed, the database management systems’ query optimizers translate the SQL text into a query plan that the database engine can execute. This plan outlines how tables are accessed, how they are joined, and the specific operations performed on the data (see Appendix Fig. 19 as an example). Inspired by the step-by-step process that database engines use to execute SQL queries, we propose a reasoning strategy to construct the final SQL output. Query plans for any given SQL query can be obtained using the “EXPLAIN" command, which provides a detailed breakdown of execution steps. However, this output is often presented in a format that is difficult to interpret by LLMs (e.g. in SQLite). To address this, we convert the output of “EXPLAIN" command into a human-readable text format that aligns more closely with the pretraining data of LLMs. The human-readable version of query plans consists of three key steps: (1) identifying and locating the relevant tables for the question, (2) performing operations such as counting, filtering, or matching between tables, and (3) delivering the final result by selecting the appropriate columns to return. This reasoning method complements the divide-and-conquer CoT strategy. While the divide-and-conquer approach is better suited for decomposing complex questions, the query plan approach excels when questions require more reasoning over the relationships between different parts of the question and the database schema. It systematically explains which tables to scan, how to match columns, and how to apply filters. Appendix Fig. 20 shows an example of a question that was answered correctly only by this method. Appendix Fig. 18 provides the prompt used for this reasoning strategy. '},
 {'heading': '4 Online Synthetic Example Generation',
  'chunk_id': 2,
  'content': ': Using M demonstrations for few-shot in-context learning has shown promising results on various related tasks (Pourreza & Rafiei, 2024a). Besides helping with specifying the task and illustrate the step-by-step process deriving the output, demonstrations constructed using relevant tables and columns can also help the model understand the underlying data schema. Based on this insight, we propose a synthetic demonstration generation strategy for Text-to-SQL – given the user question Qu, the target database D, and the selected columns ti (using a column selection approach similar to (Talaei et al., 2024)). Algorithm 2 Online Synthetic example generation strategy for Text-to-SQL. Input: User question Qu, additional user hint Hu, target database D and filtered relevant table columns t associated with the question, LLM θ, guidelines Rf for generating examples by SQL features, guidelines Rt for generating examples with filtered schema, and the numbers of examples to generate nf, nt respectively 1: P ←∅// {(qi, si) | qi, si ∈Σ∗}, where qi is input question , si is output SQL for the i-th example 2: P ←P ∪{θ(D, Rf, nf)} // Generate n examples with entire database by common SQL features 3: P ←P ∪{θ(t, Rt, nt)} // Generate examples with filtered columns to highlight correct schema usage 4: return P Algorithm 2 outlines the online synthetic example generation approach with two LLM generation steps. The first step focuses on generating illustrative examples with common SQL features described in the guideline Rf. The SQL features include equality and non-equality predicates, single table and multi-table JOIN, nested JOIN, ORDER BY and LIMIT, GROUP BY and HAVING, various aggregation functions. These are widely applicable SQL clauses and functions – the generated example SQL queries, incorporating these features, follow the BIRD SQL feature distribution (Appendix Fig 23a). The second step focuses on generating examples highlighting correct interpretation of the underlying data schema – the model θ is asked to generate examples using ti and that are similar to the examples outlined in Rt. Appendix A.10 provides the prompts used for the example generation). While a relevant example (e.g. showing a nested JOIN query with multiple tables) can be helpful for questions that require complex JOIN queries, it might also mislead the LLM for overuse (e.g. when a simple single table query is sufficient). This and the inherent ambiguity of natural language query qi, for which we draw the examples by relevance, make the example selection challenging. Thus, we generate and inject the examples to the prompt online per qi. We ask the LLM to generate many input-output pairs for in-context learning. The final set of synthetic examples for qi contains examples generated with both Rf and Rt. This ensures that the example set is diverse both in SQL features/clauses and the choice of relevant tables/columns used. The diversity of the example set is desirable to avoid over-fitting the output to certain patterns (e.g., the model always writes a SQL with JOIN if shown mostly JOIN examples). Mixing various examples for various SQL features and database tables with and without column filtering is observed to result in better generation quality overall (please see Appendix Table 8). '},
 {'heading': '3.4 Query Fixer',
  'chunk_id': 2,
  'content': ' In some cases, LLMs might generate queries that are syntactically incorrect. These queries are clear candidates for correction, as they fail to provide the correct answers. To address this, we apply an LLM-based query fixer that leverages the self-reflection (Shinn et al., 2024) method. The fixer reflects on the previously generated query, using feedback such as syntax error details or empty result sets to guide the correction process. We continue this iterative fixing approach up to a specified number of attempts, β (set to three in this paper). Appendix Fig. 21 demonstrates the prompt used for this query fixing step. '},
 {'heading': '3.5 Selection Agent',
  'chunk_id': 2,
  'content': ' With three different methods for generating SQL queries, we can generate a set of candidate queries for any given question. The key challenge in this step is selecting the correct SQL query from this pool of candidates. A naive approach would be to measure consistency among the candidates by executing them, grouping them based on their execution results, and selecting a query from the largest group as the most likely correct answer. However, this would assume that the most consistent answer is always the best one, which is not always the case. Instead, we propose a more refined picking strategy, Algorithm 3, that relies on a selection agent. Given a set of candidates SQL queries C = {c1, c2, ..., cn}, the final responses are selected by finding 5 the candidate that has the highest score assigned by the selection model. This model θp can take k candidates and rank them based on how accurately each of them answers the given question. Concretely, we formulate the selection of the final response as: cf = arg max c∈C \uf8eb \uf8ec \uf8ed (n k) X i=1 θp(ci1, . . . , cik | Qu, Hu, D) \uf8f6 \uf8f7 \uf8f8, (1) where Qu refers to the user’s question, Hu is the provided hint, and D is the target database from which the question is being asked. In Eq. 1, we pass k candidates to the selection model to be ranked, with k being between 1 and n. In the extreme case of k = 1, the model is unable to make comparisons between candidates, which complicates the evaluation process for the model. As k increases, comparing more candidates makes the process more challenging for the model, as it needs to consider different aspects simultaneously. Consequently, we set k = 2 and train a model with a classification objective to compare only two candidates at a time. Having a set of high-quality and diverse candidates, the most straightforward solution is to employ off-the-shelf LLMs to make pairwise selections. However, experiments with Gemini-1.5-pro showed that using the LLM without fine-tuning resulted in only 58.01% binary classification accuracy. This is primarily due to the candidates being very similar to one another, requiring a fine-tuned model to learn the nuances and make more accurate decisions. To train the selection agent, we first generate candidate SQL queries on the training set (of Text-to-SQL benchmarks), and group them into clusters based on their execution results. For cases where at least one cluster contains correct queries and others contains incorrect ones, we create training examples in the form of tuples (Qu, Ci, Cj, Dij, yij), where Qu is the user’s question, Ci and Cj are the two candidate queries being compared, Dij is the database schema used by both candidates, and yij ∈0, 1 is the label indicating whether Ci or Cj is the correct query. To avoid order bias during training, we randomly shuffle the order of correct and incorrect queries in each pair. Since the number of cases with both correct and incorrect candidates is limited, for instances where no correct candidate exists, we include the ground truth SQL query in the prompt as a hint to guide the model in generating correct candidates. Algorithm 3 Picking the final SQL query from a pool of candidates. Input: Set of candidate SQL queries C = {c1, c2, ..., cn}, user question Qu, hint Hu, target database D, and a selection model θp, er(ci, D) as the execution result of ci on D 1: ri ←0 for all ci ∈C // Initialize the score ri for each candidate query to zero 2: for each distinct pair (ci, cj) where i ̸= j do 3: if er(ci, D) = er(cj, D) then 4: w ←i // ci is the winner if the execution results match 5: else 6: Si,j ←schema_union(ci, cj, D) // Construct union of schemas used in ci and cj 7: w ←θp(Si,j, Qu, Hu, ci, cj)w ∈{i, j} // Use binary classifier θp to select the winner, w ∈{i, j} 8: end if 9: rw ←rw + 1 // Increase the score of the winner cw by 1 10: end for 11: cf ←arg maxci∈C ri // Select the candidate with the highest score as the final SQL query cf 12: return cf In the pseudo-code for Algorithm 3, we begin by initializing a score of zero for each candidate query. Then, for every distinct pair of queries (ci, cj), we compare both (ci, cj) and (cj, ci) to mitigate any order bias, ensuring that both candidates in a pair are fairly evaluated. If both queries produce the same execution result on the database, we mark one as the winner and increment its score, as these results suggest consistency. If the execution results differ, we generate a union of the schema used by both queries and use the binary classifier to determine which query is more likely to be correct. The classifier takes into account the question, the two candidate queries, and the combined schema to make its decision. The winner’s score is then updated accordingly. After all comparisons, the candidate with the highest score is selected as the final query. In the rare case of a tie in the final scores, we break the tie by selecting one of the candidates arbitrarily. 6 '},
 {'heading': '4 Experiments', 'chunk_id': 3, 'content': ' '},
 {'heading': '4.1 Datasets and Models',
  'chunk_id': 3,
  'content': ' We evaluate the performance of the proposed CHASE-SQL framework on two widely-recognized cross-domain datasets: BIRD (Li et al., 2024c) and Spider (Yu et al., 2018). BIRD contains over 12,751 unique question-SQL pairs from 95 large databases, spanning more than 37 professional domains, with databases designed to resemble real-world scenarios, featuring messy data rows and complex schemas. Spider consists of 10,181 questions and 5,693 unique complex SQL queries across 200 databases, covering 138 domains. The Spider dataset is divided into non-overlapping training, development, and test sets similar to BIRD. For both, we use execution accuracy (EX), the official metric for their respective leaderboard, as the primary evaluation metric to compare methods. Details of the models and their hyperparameters are provided in Appendix section A.2. '},
 {'heading': '4.2 BIRD results',
  'chunk_id': 3,
  'content': ' We present the end-to-end Text-to-SQL performance of the proposed CHASE-SQL framework using Claude- 3.5-sonnet and Gemini 1.5 pro on the BIRD development set, and Gemini 1.5 pro on the BIRD test set. We compare with both published methods (either with an available codebase and/or paper) and undisclosed methods. For a fair comparison with Gemini 1.5 pro, all LLM calls in the Claude-3.5-sonnet setting, except for the selection model, are made using Claude-3.5-sonnet (previously-trained selection model is reused). As shown in Table 2, CHASE-SQL with Gemini 1.5 pro achieves 73.01% accuracy on the BIRD development set and 73.0% on the BIRD holdout test set, outperforming all previous works and setting a new state-of-the-art performance. Table 2: Performance Comparison of different Text-to-SQL methods on BIRD benchmark. Method EX (Dev) EX (Test) Published CHASE-SQL + Gemini 1.5 (Ours) 73.01 73.0 CHASE-SQL + Claude 3.5 Sonnet (Ours) 69.53 – Distillery + GPT-4o (Maamari et al., 2024) 67.21 71.83 CHESS (Talaei et al., 2024) 65.00 66.69 MCS-SQL + GPT-4 (Lee et al., 2024) 63.36 65.45 SuperSQL (Li et al., 2024a) 58.5 62.66 Undisclosed Insights AI 72.16 70.26 AskData + GPT-4o 72.03 72.39 OpenSearch-v2 + GPT-4o 69.3 72.28 PURPLE-RED + GPT-4o 68.12 70.21 Arcwise + GPT-4o 67.99 66.21 ExSL + granite-34b-code 67.47 67.76 Table 3: Performance Comparison of different Text-to-SQL methods on Spider test set. Method EX Training with Spider MCS-SQL + GPT-4 (Lee et al., 2024) 89.6 ✓ CHASE-SQL + Gemini 1.5 (Ours) 87.6 ✗ CHESS (Talaei et al., 2024) 87.2 ✗ DAIL-SQL + GPT-4 (Gao et al., 2023) 86.6 ✓ DIN-SQL + GPT-4 (Pourreza & Rafiei, 2024a) 85.3 ✓ C3 + ChatGPT (Dong et al., 2023) 82.3 ✓ RESDSQL 3B (Li et al., 2023a) 79.9 ✓ DIN-SQL + CodeX (Pourreza & Rafiei, 2024a) 78.2 ✓ T5-3B+NatSQL (Rai et al., 2023) 78.0 ✓ Graphix-3B+PICARD (Li et al., 2023b) 77.6 ✓ '},
 {'heading': '4.3 Spider results',
  'chunk_id': 4,
  'content': ' We assess the generalizability of the proposed CHASE-SQL by evaluating it in an end-to-end way on the Spider test set without modifying the few-shot samples in the prompts or training a new selection model, i.e. without using and data from the target distribution. This approach allows us to test the performance of CHASE-SQL on different unseen query and database distributions compared to the data from training distributions. Table 3 demonstrates that CHASE-SQL achieves an execution accuracy of 87.6% on the Spider test set, placing it second among methods that have undergone specific training or prompt optimization for the Spider dataset. This highlights the strong generalizability of CHASE-SQL and its potential for generating high quality Text-to-SQL for unseen samples coming from very different distributions and unique challenges. 7 '},
 {'heading': '4.4 Generator and selection performance',
  'chunk_id': 4,
  'content': ' Table 4: Ablation studies on single candidate gen- eration performance of the candidate generators compared with original BIRD prompt + zero-shot CoT with Gemini 1.5 pro on the BIRD dev set. Selector is our final performance applying the se- lection agent to 21 candidates generated from all generators. Method Execution Accuracy (%) ∆(%) Baseline 57.75 - QP CoT 63.62 +5.87 DC CoT 63.92 +6.17 OS CoT 67.09 +9.34 Baseline w Query Fixer 61.58 +3.83 QP CoT w Query Fixer 65.51 +7.76 DC CoT w Query Fixer 65.77 +8.02 OS CoT w Query Fixer 68.02 +10.27 Generator + Fixer: To reveal performance of genera- tors, we conducted an ablation study to evaluate the per- formance of each candidate generation method before and after applying the query fixer. We compare the performance of the proposed generators in producing a single candidate query against the original BIRD prompt (Li et al., 2024c), augmented with zero-shot CoT reasoning (Kojima et al., 2022), which serves as the baseline for assessing the quality of prompts. The results, shown in Table 4, indicate that the proposed methods significantly improve SQL generation performance, compared to the naive baseline, towards the goal of producing high-quality candidates while maintain- ing diversity. Among the candidate generators, the online synthetic data generation approach produced an impressive performance of 68.02%, demonstrating its effectiveness in leveraging test-time compute to improve LLM performance by generating high-quality synthetic examples. Furthermore, the query fixer proved crucial, enhancing the quality of the candidate pool and increasing performance by nearly 2% across all candidate generators. Table 5: Evaluating the binary selection accuracy of the different selection models. '},
 {'heading': 'Selection',
  'chunk_id': 5,
  'content': ' Model Binary Acc. (%) Claude-3.5-sonnet 60.21 Gemini-1.5-pro 63.98 Tuned Gemma 2 9B 64.28 Tuned Gemini-1.5-flash 71.01 Selection: We conducted an analysis on the binary selection accu- racy of the selection agent for cases where, in a pairwise comparison, one candidate is correct and the other is incorrect. We exclude cases where both candidates are either correct or incorrect, as the selection would not affect the outcome since both candidates have the same label. We compare the performance of Claude-3.5-sonnet and Gemini-1.5-pro (both out-of-the-box without fine-tuning) with two fine-tuned models: 1) Gemma 2 9B and 2) Gemini-1.5-flash. As shown in Table 5, both fine-tuned models achieve higher accuracy than the untuned counterparts, demonstrating the importance of fine-tuning to teach the model about the specific preferences. '},
 {'heading': 'Candidate Generation Analysis',
  'chunk_id': 5,
  'content': ': We analyze the performance of each candidate generator method individually. To better understand the performance potential when effectively selecting the correct SQL query from the candidate pool, we generate seven candidate SQL queries from each generator method (21 candidates in total) for all samples in the BIRD development set. We determine this number of candidates based on the observation that increasing the candidate pool beyond 20 did not yield significant improvements, as illustrated in Fig. 2d. By assuming access to an oracle selection model that always selects the correct SQL query from the seven candidates, we calculate the upper-bound performance achievable for each generator. Conversely, by assuming an adversarial selection model that always selects the wrong SQL query, we determine the lower-bound performance. Fig. 2 illustrates the upper-bound and lower-bound performance for all three methods together with the performance of our selection agent. As shown, the upper-bound performance of the two different CoT methods is generally higher than that of the synthetic example generation method for different number of candidates. However, their lower-bound performance is also lower than the synthetic method. Lower-bound accuracy reflects cases where all candidates are correct, reducing the noise in the selection process since it doesn’t matter which candidate is chosen, so a higher lower-bound is preferred. This is evident in the selection agent’s performance, where a drop in the lower bound leads to diminishing returns from increasing the upper bound, causing the selection agent’s performance to plateau. Additionally, the upper-bound performance of combining all three methods reaches 82.79%, highlighting the significant room for improvement through better candidate picking methods. This demonstrates that the LLM’s parametric knowledge already contains the information needed to solve most questions, highlighting the need for ensemble approaches to effectively extract and utilize this knowledge. 8 (a) Upper-bound and lower-bound Accuracy for Divide and Conquer CoT (b) Upper-bound and lower-bound Accuracy for Online Synthetic Example (c) Upper-bound and lower-bound performance for Query Plan CoT. (d) Upper-bound performance of all three can- didate generators across different difficulty cat- egories. Figure 2: Comparison of the upper- and lower-bound performance of different candidate generators. Additionally, we evaluate the upper-bound performance by combining all candidates from three candidate generation methods across the simple, moderate, and challenging difficulty levels for the BIRD development set. These difficulty categories are assigned by human experts during the creation of the BIRD development set. Fig. 2d shows that, as expected, the upper-bound performance increases with the number of candidates across all difficulty levels. However, for the challenging and moderate classes, the improvement plateaus earlier than in the simple class, suggesting that generating more samples does not further improve the upper-bound performance for these two difficulty levels. Fig. 2 presents a Venn diagram showcasing the performance of three generation methods: Query Plan, Divide and Conquer, and with Synthetic Examples. The numbers within the intersecting regions represent the instances where multiple methods generated at least one correct candidate. This diagram visually highlights the unique contributions of each method, which indicates the necessity of using all three generators. Additionally, in Fig. 3b, we compare the number of correct queries generated by each SQL generation method that are not correct by the other generators. The divide-and-conquer approach outperforms the others on challenging questions, while the query plan method excels on moderately difficult queries. To further analyze the performance of the generators across different domains and varying numbers of columns and tables, we compare the number of correct queries generated for each database, as shown in Appendix Fig. 4. As illustrated, both CoT methods generally perform similarly across databases, while the online synthetic example generation method significantly increases diversity, resulting in more correct answers overall across different databases. '},
 {'heading': 'Selection Agent Analysis',
  'chunk_id': 5,
  'content': ': We evaluate the query-picking performance by comparing the Text-to-SQL execution accuracy of the selection agent with the self-consistency method (using majority voting) Wang et al. (2022), an oracle model (upper bound), and an adversarial model (lower bound). To conduct the evaluation, we generate 10 samples from each candidate generation method using two different sampling temperatures: 0.5 and 1.8. The results, shown in Table 6, demonstrate that the selection agent significantly outperforms the self-consistency method with a large margin, roughly 6%. As expected, increasing the sampling temperature 9 35 30 33 38 72 23 1045 '},
 {'heading': 'Query Plan', 'chunk_id': 6, 'content': ' '},
 {'heading': 'Synthetic Example', 'chunk_id': 6, 'content': ' '},
 {'heading': 'Divide and Conquer',
  'chunk_id': 6,
  'content': ' Unsolved Questions: 258 (a) Venn diagram illustrating the num- ber of instances for which each method: Query Plan, Synthetic Example, Divide and Conquer, produces at least one cor- rect candidate. The overlap regions repre- sent multiple methods generating correct candidates. (b) Number of correct queries across different complexity levels that were answered by each method. Figure 3: Comparison of SQL generation methods: Venn diagram showing unique and overlapping correct answers (left) and the performance across different complexity levels (right). raises the upper bound but also lowers the lower bound. This effect is more pronounced for the synthetic data generation method compared to the two CoT methods, mainly because LLMs generate reasoning steps before producing the final SQL query, which helps mitigate the randomness introduced by high-temperature sampling. The performance with self-consistency method generally decreases as temperature increases, since the majority cluster becomes smaller with more random queries. However, the proposed trained selection agent is less affected by temperature scaling and, in two cases, even improved its performance with a more diverse pool of samples. Table 6: Performance comparison of different picking methods on the candidates generated by the candidate generators on BIRD development set with two different temperatures. QP refers to query plan COT, DC refers to divide and conquer COT, and OS is the online synthetic example generation method. Picking Method QP (T=0.5) QP (T=1.8) DC (T=0.5) DC (T=1.8) OS (T=0.5) OS (T=1.8) Lower Bound 50.46 48.63 51.37 47.39 60.43 50.98 Upper Bound 78.55 80.44 78.42 79.34 74.77 79.66 Self-consistency 65.78 65.51 66.43 64.41 67.34 66.88 Our Selection Agent 71.7 71.73 71.31 70.53 70.4 71.38 '},
 {'heading': '4.5 Ablation Studies',
  'chunk_id': 6,
  'content': ' In the previous sections, we evaluate the importance of the selection agent and each candidate generation method. Next, we focus on the analysis of the remaining components of CHASE-SQL: LSH for value retrieval, the query fixer, and three reasoning strategies (QP, OS, and DC). Table 7 shows the performance of CHASE- SQL without each of these steps, highlighting their significance in achieving higher-quality performance. The results demonstrate the contribution of each component, where removing LSH, the query fixer, or any of the candidate generators leads to a reduction in execution accuracy, further validating the importance of these components of CHASE-SQL. Moreover, the table compares the performance of our binary selection agent with two other selection methods: self-consistency (Wang et al., 2022) and a ranker agent. The ranker agent receives all candidates generated by our three candidate generators in a single prompt, compares them, and produce a ranking for each. For the ranker agent, we select the query with the lowest rank as the best answer. The binary selection agent significantly outperforms both the self-consistency and ranker agents, demonstrating the effectiveness of the proposed method. 10 '},
 {'heading': '5 Conclusion',
  'chunk_id': 6,
  'content': ' Table 7: Ablation studies on the performance of CHASE- SQL after removing the query fixer, LSH for value re- trieval, and reasoning strategies, i.e., QP, OS, and DC. Method Execution Accuracy (%) ∆(%) CHASE-SQL All 73.01 - CHASE-SQL w self-consistency 68.84 -4.17 CHASE-SQL w ranker agent 65.51 -7.5 CHASE-SQL w/o LSH 70.09 -2.92 CHASE-SQL w/o Query Fixer 69.23 -3.78 CHASE-SQL w/o QP 72.36 -0.65 CHASE-SQL w/o OS 72.16 -0.85 CHASE-SQL w/o DC 71.77 -1.24 We introduce a novel agentic framework, CHASE-SQL, to leverage test-time compute for generating diverse, high-quality SQL queries and accurately selecting the correct one. We propose multiple chain-of-thought prompting methods and an online synthetic example generation technique, along with a query selection mechanism that scores candidates based on pairwise comparisons. Our framework, CHASE-SQL, sets a new state-of-the-art in the notable public Text-to-SQL leaderboard (at the time of the submission), demon- strating the effectiveness of test-time computation for both generating diverse queries and selecting the most accurate response. CHASE-SQL addresses key issues like query diversity and selection optimization, paving the way for further improvements in complex reasoning tasks encountered at real-world Text-to-SQL challenges. '},
 {'heading': 'Acknowledgments',
  'chunk_id': 7,
  'content': ' We would like to thank Per Jacobsson, Raj Sinha, Zeke Miller, Reza Sherkat, James Su, Zhixian Yan, David Culler, and Xiance Si, for their valuable comments and feedbacks on our paper. We would also like to thank the BIRD team for their invaluable assistance with the evaluation of the BIRD test set. '},
 {'heading': 'References',
  'chunk_id': 7,
  'content': ' Ion Androutsopoulos, Graeme D Ritchie, and Peter Thanisch. Natural language interfaces to databases–an introduction. Natural language engineering, 1(1):29–81, 1995. Ruichu Cai, Jinjie Yuan, Boyan Xu, and Zhifeng Hao. Sadga: Structure-aware dual graph aggregation network for text-to-sql. Advances in Neural Information Processing Systems, 34:7664–7676, 2021. Ruisheng Cao, Lu Chen, Zhi Chen, Yanbin Zhao, Su Zhu, and Kai Yu. Lgesql: line graph enhanced text-to-sql model with mixed local and non-local relations. arXiv preprint arXiv:2106.01093, 2021. Mark Chen, Jerry Tworek, Heewoo Jun, Qiming Yuan, Henrique Ponde de Oliveira Pinto, Jared Kaplan, Harri Edwards, Yuri Burda, Nicholas Joseph, Greg Brockman, et al. Evaluating large language models trained on code. arXiv preprint arXiv:2107.03374, 2021. Xinyun Chen, Maxwell Lin, Nathanael Schärli, and Denny Zhou. Teaching large language models to self-debug. arXiv preprint arXiv:2304.05128, 2023. DongHyun Choi, Myeong Cheol Shin, EungGyun Kim, and Dong Ryeol Shin. Ryansql: Recursively applying sketch-based slot fillings for complex text-to-sql in cross-domain databases. Computational Linguistics, 47 (2):309–332, 2021. Mayur Datar, Nicole Immorlica, Piotr Indyk, and Vahab S Mirrokni. Locality-sensitive hashing scheme based on p-stable distributions. In Proceedings of the twentieth annual symposium on Computational geometry, pp. 253–262, 2004. Xuemei Dong, Chao Zhang, Yuhang Ge, Yuren Mao, Yunjun Gao, Jinshu Lin, Dongfang Lou, et al. C3: Zero-shot text-to-sql with chatgpt. arXiv preprint arXiv:2307.07306, 2023. Dawei Gao, Haibin Wang, Yaliang Li, Xiuyu Sun, Yichen Qian, Bolin Ding, and Jingren Zhou. Text-to-sql empowered by large language models: A benchmark evaluation. arXiv preprint arXiv:2308.15363, 2023. Jonathan Herzig, Paweł Krzysztof Nowak, Thomas Müller, Francesco Piccinno, and Julian Martin Eisenschlos. Tapas: Weakly supervised table parsing via pre-training. arXiv preprint arXiv:2004.02349, 2020. 11 Wonseok Hwang, Jinyeong Yim, Seunghyun Park, and Minjoon Seo. A comprehensive exploration on wikisql with table-aware word contextualization. arXiv preprint arXiv:1902.01069, 2019. George Katsogiannis-Meimarakis and Georgia Koutrika. A survey on deep learning approaches for text-to-sql. The VLDB Journal, 32(4):905–936, 2023. Takeshi Kojima, Shixiang Shane Gu, Machel Reid, Yutaka Matsuo, and Yusuke Iwasawa. Large language models are zero-shot reasoners. Advances in neural information processing systems, 35:22199–22213, 2022. Dongjun Lee, Choongwon Park, Jaehyuk Kim, and Heesoo Park. Mcs-sql: Leveraging multiple prompts and multiple-choice selection for text-to-sql generation. arXiv preprint arXiv:2405.07467, 2024. Boyan Li, Yuyu Luo, Chengliang Chai, Guoliang Li, and Nan Tang. The dawn of natural language to sql: Are we fully ready? arXiv preprint arXiv:2406.01265, 2024a. Fei Li and Hosagrahar V Jagadish. Constructing an interactive natural language interface for relational databases. Proceedings of the VLDB Endowment, 8(1):73–84, 2014. Haoyang Li, Jing Zhang, Cuiping Li, and Hong Chen. Resdsql: Decoupling schema linking and skeleton parsing for text-to-sql. In Proceedings of the AAAI Conference on Artificial Intelligence, volume 37, pp. 13067–13075, 2023a. Haoyang Li, Jing Zhang, Hanbing Liu, Ju Fan, Xiaokang Zhang, Jun Zhu, Renjie Wei, Hongyan Pan, Cuiping Li, and Hong Chen. Codes: Towards building open-source language models for text-to-sql. Proceedings of the ACM on Management of Data, 2(3):1–28, 2024b. Jinyang Li, Binyuan Hui, Reynold Cheng, Bowen Qin, Chenhao Ma, Nan Huo, Fei Huang, Wenyu Du, Luo Si, and Yongbin Li. Graphix-t5: Mixing pre-trained transformers with graph-aware layers for text-to-sql parsing. arXiv preprint arXiv:2301.07507, 2023b. Jinyang Li, Binyuan Hui, Ge Qu, Jiaxi Yang, Binhua Li, Bowen Li, Bailin Wang, Bowen Qin, Ruiying Geng, Nan Huo, et al. Can llm already serve as a database interface? a big bench for large-scale database grounded text-to-sqls. Advances in Neural Information Processing Systems, 36, 2024c. Yujia Li, David Choi, Junyoung Chung, Nate Kushman, Julian Schrittwieser, Rémi Leblond, Tom Eccles, James Keeling, Felix Gimeno, Agustin Dal Lago, et al. Competition-level code generation with alphacode. Science, 378(6624):1092–1097, 2022. Karime Maamari, Fadhil Abubaker, Daniel Jaroslawicz, and Amine Mhedhbi. The death of schema linking? text-to-sql in the age of well-reasoned language models. arXiv preprint arXiv:2408.07702, 2024. Ansong Ni, Srini Iyer, Dragomir Radev, Veselin Stoyanov, Wen-tau Yih, Sida Wang, and Xi Victoria Lin. Lever: Learning to verify language-to-code generation with execution. In International Conference on Machine Learning, pp. 26106–26128. PMLR, 2023. Rubén Pérez-Mercado, Antonio Balderas, Andrés Muñoz, Juan Francisco Cabrera, Manuel Palomo-Duarte, and Juan Manuel Dodero. Chatbotsql: Conversational agent to support relational database query language learning. SoftwareX, 22:101346, 2023. Mohammadreza Pourreza and Davood Rafiei. Din-sql: Decomposed in-context learning of text-to-sql with self-correction. Advances in Neural Information Processing Systems, 36, 2024a. Mohammadreza Pourreza and Davood Rafiei. Dts-sql: Decomposed text-to-sql with small large language models. arXiv preprint arXiv:2402.01117, 2024b. Mohammadreza Pourreza, Ruoxi Sun, Hailong Li, Lesly Miculicich, Tomas Pfister, and Sercan O Arik. Sql-gen: Bridging the dialect gap for text-to-sql via synthetic data and model merging. arXiv preprint arXiv:2408.12733, 2024. 12 Abdul Quamar, Vasilis Efthymiou, Chuan Lei, and Fatma Özcan. Natural language interfaces to data. Found. Trends Databases, 11(4):319–414, 2022. doi: 10.1561/1900000078. URL https://doi.org/10. 1561/1900000078.'}]

In [2]:
from t2j.prem_sdk import PremSDK  
from t2j.prompts import Prompts
from t2j.chunker import DocumentChunker
from t2j.utils import *

import json

N = 100
promptsClass = Prompts()
model = PremSDK()
chunker = DocumentChunker(prompts=promptsClass, model=model)

mapping, headings = chunker.create_content_mapping(final_headings)

In [3]:
mapping['4']

' Databases might contain very high number of rows, with often only a few being relevant to a query. Retrieving relevant values is crucial as they can be used in various SQL clauses like ‘WHERE’ and ‘HAVING’. Similar to the approach in (Talaei et al., 2024), we begin by extracting keywords from the given question using an LLM prompted with few-shot examples. For each keyword, we employ locality-sensitive hashing (LSH) (Datar et al., 2004) to retrieve the most syntactically-similar words, and re-rank them based on embedding-based similarity and edit distance. This approach is robust to typos in the question and considers keyword semantics during retrieval. '

In [4]:
res = chunker.create_raw_json(headings)

In [7]:
print(json.dumps(res, indent=2))

{
  "1 Introduction": {
    "content": "0",
    "sub-headings": []
  },
  "2 Related Work": {
    "content": "1",
    "sub-headings": []
  },
  "3 Methods": {
    "content": "2",
    "sub-headings": [
      {
        "3.1 Overall Framework": {
          "content": "3",
          "sub-headings": []
        }
      },
      {
        "3.2 Value Retrieval": {
          "content": "4",
          "sub-headings": []
        }
      },
      {
        "3.3 Multi-path Candidate Generation": {
          "content": "5",
          "sub-headings": [
            {
              "Divide and Conquer CoT": {
                "content": "6",
                "sub-headings": []
              }
            },
            {
              "Query Plan CoT": {
                "content": "7",
                "sub-headings": []
              }
            }
          ]
        }
      },
      {
        "3.4 Query Fixer": {
          "content": "9",
          "sub-headings": []
        }
      },
      {
       

In [11]:
def replace_content(data, content_map):
    for key, value in data.items():
        # Replace content
        if 'content' in value:
            content_id = value['content']
            if content_id in content_map:
                value['content'] = content_map[content_id]

        # Recurse into sub-headings
        if 'sub-headings' in value and isinstance(value['sub-headings'], list):
            for sub_item in value['sub-headings']:
                for sub_key, sub_value in sub_item.items():
                    replace_content({sub_key: sub_value}, content_map)
                    
res


{'1 Introduction': {'content': ' Text-to-SQL, as a bridge between human language and machine-readable structured query languages, is crucial for many use cases, converting natural language questions into executable SQL commands (Androutsopoulos et al., 1995; Li & Jagadish, 2014; Li et al., 2024c; Yu et al., 2018; ?). By enabling users to interact with complex database systems without requiring SQL proficiency, Text-to-SQL empowers users to extract valuable insights, perform streamlined data exploration, make informed decisions, generate data-driven reports and mine better features for machine learning (Chen et al., 2023; Pérez-Mercado et al., 2023; Pourreza & Rafiei, 2024a; Pourreza et al., 2024; Sun et al., 2023; Wang et al., 2019; Xie et al., 2023). Furthermore, Text-to-SQL systems play a pivotal role in automating data analytics with complex reasoning and powering conversational agents, expanding their applications beyond traditional data retrieval (Sun et al., 2023; Xie et al., 202

In [9]:
ASDASD

In [1]:
import json

from t2j.decomposer import SchemaDecomposer  # adjust import if needed

SCHEMA_PATH = r"C:\Users\Pratyush\Desktop\text-to-json\examples\text2SQL-solutions\schema.json"

with open(SCHEMA_PATH, 'r') as f:
    schema_dict = json.load(f)

sd = SchemaDecomposer(schema_dict)
res = sd.decompose()
sd.print_schema_tree(res)

 trunk 0  title
 trunk 0  related_work[]
 trunk 0  authors[]
 trunk 0  abstract
 trunk 0  approach[]
branch 1 ──── approach_name
branch 1 ──── description
branch 1 ──── steps[]
branch 2 ──────── step_title
branch 2 ──────── details
branch 1 ──── improvements
branch 2 ──────── metric
branch 2 ──────── value_added
 trunk 0  dataset[]
branch 1 ──── name
branch 1 ──── source
branch 1 ──── preprocessing
branch 2 ──────── steps
branch 2 ──────── tools_used
 trunk 0  experiment_results[]
branch 1 ──── experiment_name
branch 1 ──── metrics[]
branch 2 ──────── metric_name
branch 2 ──────── value
 trunk 0  references[]


In [2]:
res

[{'path': 'title',
  'type': 'string',
  'description': 'Title of the paper or document',
  'node_type': 'trunk'},
 {'path': 'related_work[]',
  'type': 'array<string>',
  'description': 'List of strings, with related works done; keep the points short and precise',
  'node_type': 'trunk'},
 {'path': 'authors[]',
  'type': 'array<string>',
  'description': 'List of authors of the paper',
  'node_type': 'trunk'},
 {'path': 'abstract',
  'type': 'string',
  'description': 'Up to 5 bullet points describing what is new in the approach',
  'node_type': 'trunk'},
 {'path': 'approach[]',
  'type': 'array<object>',
  'description': 'List of methods used in the approach',
  'node_type': 'trunk'},
 {'path': 'approach[].approach_name',
  'type': 'string',
  'description': 'Name of the method or technique',
  'node_type': 'branch'},
 {'path': 'approach[].description',
  'type': 'string',
  'description': 'Brief summary of what the method is',
  'node_type': 'branch'},
 {'path': 'approach[].steps[]'

In [2]:
def max_child_depth(schema_list, trunk_path):
    def get_depth(path):
        return path.count("[]") + path.count(".")

    trunk_path = trunk_path.rstrip('.')

    max_depth = 0
    for field in schema_list:
        path = field['path']
        if path == trunk_path:
            continue
        if path.startswith(trunk_path + ".") or path.startswith(trunk_path + "[]"):
            relative_path = path[len(trunk_path):].lstrip('.').lstrip('[]')
            depth = get_depth(relative_path)
            max_depth = max(max_depth, depth)

    return max_depth

for r in res:
    print(max_child_depth(res, r['path']), r)

0 {'path': 'title', 'type': 'string', 'description': 'Title of the paper or document', 'node_type': 'trunk'}
0 {'path': 'related_work[]', 'type': 'array<string>', 'description': 'List of strings, with related works done; keep the points short and precise', 'node_type': 'trunk'}
0 {'path': 'authors[]', 'type': 'array<string>', 'description': 'List of authors of the paper', 'node_type': 'trunk'}
0 {'path': 'abstract', 'type': 'string', 'description': 'Up to 5 bullet points describing what is new in the approach', 'node_type': 'trunk'}
2 {'path': 'approach[]', 'type': 'array<object>', 'description': 'List of methods used in the approach', 'node_type': 'trunk'}
0 {'path': 'approach[].approach_name', 'type': 'string', 'description': 'Name of the method or technique', 'node_type': 'branch'}
0 {'path': 'approach[].description', 'type': 'string', 'description': 'Brief summary of what the method is', 'node_type': 'branch'}
0 {'path': 'approach[].steps[]', 'type': 'array<object>', 'description':

In [3]:
# sd = SchemaDecomposer(chunks)
# chunks = sd.decompose()
# sd.print_schema_tree(chunks)

NameError: name 'chunks' is not defined

In [3]:
# remove content
chunks = {'1 Introduction': {'content': ' Text-to-SQL, as a bridge between human language and machine-readable structured query languages, is crucial for many use cases, converting natural language questions into executable SQL commands (Androutsopoulos et al., 1995; Li & Jagadish, 2014; Li et al., 2024c; Yu et al., 2018; ?). By enabling users to interact with complex database systems without requiring SQL proficiency, Text-to-SQL empowers users to extract valuable insights, perform streamlined data exploration, make informed decisions, generate data-driven reports and mine better features for machine learning (Chen et al., 2023; Pérez-Mercado et al., 2023; Pourreza & Rafiei, 2024a; Pourreza et al., 2024; Sun et al., 2023; Wang et al., 2019; Xie et al., 2023). Furthermore, Text-to-SQL systems play a pivotal role in automating data analytics with complex reasoning and powering conversational agents, expanding their applications beyond traditional data retrieval (Sun et al., 2023; Xie et al., 2023). As data continues to grow exponentially, the ability to query databases efficiently without extensive SQL knowledge becomes increasingly vital for a broad range of applications. Text-to-SQL can be considered a specialized form of code generation, with the contextual information potentially including the database schema, its metadata and along with the values. In the broader code generation domain, utilizing LLMs to generate a wide range of diverse candidates and select the best one has proven to be effective (Chen et al., 2021; Li et al., 2022; Ni et al., 2023). However, it is non-obvious what 1 arXiv:2410.01943v1  [cs.LG]  2 Oct 2024 leads to most effective candidate proposal and winner selector mechanisms. A straightforward yet effective approach involves generating candidates using zero-/few-shot or open-ended prompting, followed by selecting the best options utilizing self-consistency (Wang et al., 2022), which entails clustering candidates based on their execution outputs. This approach has demonstrated promising results in several studies (Lee et al., 2024; Maamari et al., 2024; Talaei et al., 2024; Wang et al., 2023). However, a single prompt design might not fully unleash the extensive Text-to-SQL knowledge of LLMs, and self-consistency methods might not be always effective. In fact, as illustrated in Table 1, the most consistent answers would not always be the correct ones, with an upper-bound performance 14% higher than that achieved through self-consistency. This substantial gap highlights the potential for significant improvement by implementing more effective selection methods to identify the best answer from the pool of candidate queries. Table 1: Evaluating single-query gen- eration vs. ensemble methods of self- consistency and the upper bound that can be achieved for Text-to-SQL with Gemini 1.5 Pro on the BIRD dev set. EX stands for execution accuracy. Method EX (%) Single query 63.01 Self-consistency 68.84 (+ 5.84) Upper-bound 82.79 (+ 19.78) Building on the challenges outlined in the previous section, we pro- pose novel approaches to improve LLM performance for Text-to-SQL by leveraging judiciously-designed test-time computations in an agentic framework. As indicated by the upper bound in Table 1, utilizing LLMs’ intrinsic knowledge offers significant potential for improvement. We propose methods that generate a diverse set of high-quality can- didate responses and apply a selection mechanism to identify the best answer. Achieving both high-quality and diverse candidate responses is critical for the success of scoring-based selection methods. Low diversity limits improvement potential and reduces the difference be- tween self-consistency and scoring-based approaches. While techniques like increasing temperature or reordering prompt contents can boost diversity, they often compromise the quality of the candidates. To address this, we introduce effective candidate generators designed to enhance diversity while maintaining high-quality outputs. Specifically, we propose three distinct candidate generation approaches, each capable of producing high-quality responses. The first is inspired by the divide-and-conquer algorithm, which breaks down complex problems into smaller, manageable parts to handle difficult queries. The second employs a query execution-plan-based chain-of-thought strategy, where the reasoning process mirrors the steps a database engine takes during query execution. Lastly, we introduce a novel online synthetic example generation method, which helps the model better understand the underlying data schema of the test database. These methods, when used independently, can produce highly-accurate SQL outputs. To effectively select the best answer among candidates, we introduce a selection agent, trained with a classification objective, that assigns scores based on pairwise comparisons between candidate queries. With this agent, we construct a comparison matrix for all candidates and select the final response based on the highest cumulative score. By combining these candidate generation methods with the proposed scoring model, we create an ensemble approach that leverages the strengths of each strategy to significantly improve overall performance. We present comprehensive evaluations on the efficacy of proposed methodologies of CHASE-SQL. Our innovative candidate generation approaches demonstrate superior performance compared to traditional generic CoT prompts, illustrating their capability in guiding LLMs through the decomposition of complex problems into manageable intermediate steps. Furthermore, the proposed selection agent significantly outperforms conventional consistency-based methods, contributing to the state-of-the-art results. Specifically, CHASE- SQL reaches an execution accuracy of 73.01% and 73.0% on the development set and test set of the challenging BIRD Text-to-SQL dataset which outperforms all of the published and undisclosed methods on this benchmark, by a large margin. ',
  'sub-headings': []},
 '2 Related Work': {'content': ' Early Text-to-SQL methods predominantly utilized sequence-to-sequence architectures, encoding user queries and database schemas using models such as Graph Neural Networks (GNNs), Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, and pre-trained transformer encoders (Cai et al., 2021; Cao et al., 2021; Hwang et al., 2019). On the decoding side, these systems employed either slot-filling or auto-regressive modelling approaches to construct the final SQL queries from the encoded inputs (Choi et al., 2021; Wang et al., 2019). Additionally, tabular language models like TaBERT (Yin et al., 2020), 2 TaPas (Herzig et al., 2020), and Grappa (Yu et al., 2020) have been developed to encode both tables and textual data effectively. However, the landscape has evolved with the widespread use of LLMs, which have largely replaced earlier methods with their superior performance (Katsogiannis-Meimarakis & Koutrika, 2023; Quamar et al., 2022). Initially, efforts concentrated on optimizing prompt designs for these LLMs (Dong et al., 2023; Gao et al., 2023; Pourreza & Rafiei, 2024a). Subsequent advancements have introduced more complex methodologies, including schema linking (Li et al., 2024b; Pourreza & Rafiei, 2024a,b; Talaei et al., 2024), self-correction or self-debugging (Chen et al., 2023; Talaei et al., 2024; Wang et al., 2023), and self-consistency techniques (Lee et al., 2024; Maamari et al., 2024; Sun et al., 2023; Talaei et al., 2024), further enhancing the performance by proposing complex LLM-based pipelines. ',
  'sub-headings': []},
 '3 Methods': {'content': ' ',
  'sub-headings': [{'3.1 Overall Framework': {'content': ' This section outlines the proposed CHASE-SQL framework, which consists of four primary components: 1) Value retrieval, 2) Candidate generator, 3) Query fixer, and 4) Selection agent. As illustrated in Fig. 1. The proposed framework begins by retrieving relevant database values. Subsequently, all contextual information, including retrieved values, database metadata, and schema, is provided to an LLM to generate candidate queries. These candidate queries then undergo a fixing loop, and finally, all candidates are compared in a pairwise way using the trained selection agent to pick the correct answer. The following sections delve into the details of each component. Figure 1: Overview of the proposed CHASE-SQL framework for Text-to-SQL, with value retrieval and using a selection agent for improve picking of the answers among the generated candidates along with a fixer to provide feedback for refinement of the outputs. ',
     'sub-headings': []}},
   {'3.2 Value Retrieval': {'content': ' Databases might contain very high number of rows, with often only a few being relevant to a query. Retrieving relevant values is crucial as they can be used in various SQL clauses like ‘WHERE’ and ‘HAVING’. Similar to the approach in (Talaei et al., 2024), we begin by extracting keywords from the given question using an LLM prompted with few-shot examples. For each keyword, we employ locality-sensitive hashing (LSH) (Datar et al., 2004) to retrieve the most syntactically-similar words, and re-rank them based on embedding-based similarity and edit distance. This approach is robust to typos in the question and considers keyword semantics during retrieval. ',
     'sub-headings': []}},
   {'3.3 Multi-path Candidate Generation': {'content': ' As shown in Table 1, relying solely on consistency among responses can lead to sub-optimal performance. Therefore, we prioritize diversity in generation of multiple response candidates to increase the likelihood of generating at least one correct answer. Among the diverse responses generated by the candidate generators, we select one as the final response using a selection agent that compares candidates pairwise. To generate 3 diverse responses, we increase the next token sampling temperature, and also shuffle the order of columns and tables in the prompt. Chain-of-Thought (CoT) prompting (Wei et al., 2022) has been proposed to enhance LLMs’ reasoning abilities by conditioning their final responses on a step-by-step chain of reasoning. Most CoT prompting approaches rely on few-shot examples in the prompt to guide LLMs on thinking step-by-step, following the format M = (qi, ri, si), where qi is the example question, ri is the reasoning path, and si is the ground truth SQL query for qi. We employ two distinct reasoning methods and an online synthetic example generation approach. As shown in Fig. 3a, different generators can yield different outputs, indicating their effectiveness for specific questions and databases. ',
     'sub-headings': [{'Divide and Conquer CoT': {'content': ': Divide-and-conquer perspective brings breaking down complex problems into smaller sub-problems, solving each individually, and then combining the solutions to obtain the final answer. Along these lines, we propose a CoT prompting approach that first decomposes the given question into smaller sub-problems using pseudo-SQL queries. In the ’conquer’ step, the solutions to these sub-problems are aggregated to construct the final answer. Finally, an optimization step is applied to the constructed query to remove redundant clauses and conditions. This approach is particularly powerful handling complex scenarios that involve nested queries, e.g. intricate WHERE or HAVING conditions, and queries requiring advanced mathematical operations. In Appendix Fig. 17, we exemplify a question and its corresponding SQL query that was successfully solved using this generator, a scenario the other methods considered in this paper could not address due to the query’s complex conditions and SQL clauses. For a more detailed view of the divide-and-conquer prompt, please see Appendix Fig. 16. Additionally, Alg. 1 outlines the step-by-step process of this strategy to generate the final SQL output using a single LLM call. Algorithm 1 Divide and Conquer Chain-of-Thought (CoT) Strategy for Text-to-SQL. Input: Set of human-annotated few-shot examples M, user question Qu, target database D associated with the question, and a large language model (LLM) θ. Divide: 1: Sq ←θ(M, D, Qu) // Decompose the original question Qu into a set of sub-questions Sq 2: Ssql ←∅// Initialize an empty set Ssql to store partial SQL queries for each sub-question Conquer: 3: for each sub-question qi in Sq do 4: // Generate a partial SQL query for each sub-question qi 5: Ssql ←Ssql ∪{θ(M, D, Qu, q1, ..., qi, sql1, ..., sqli−1)} 6: end for Assemble: 7: Sf ←θ(M, D, Qu, Sq, Ssql) // Assemble the final SQL query Sf from all sub-queries in Ssql 8: return Sf ',
        'sub-headings': []}},
      {'Query Plan CoT': {'content': ': A query (execution) plan is a sequence of steps that the database engine follows to access or modify the data described by a SQL command. When a SQL query is executed, the database management systems’ query optimizers translate the SQL text into a query plan that the database engine can execute. This plan outlines how tables are accessed, how they are joined, and the specific operations performed on the data (see Appendix Fig. 19 as an example). Inspired by the step-by-step process that database engines use to execute SQL queries, we propose a reasoning strategy to construct the final SQL output. Query plans for any given SQL query can be obtained using the “EXPLAIN" command, which provides a detailed breakdown of execution steps. However, this output is often presented in a format that is difficult to interpret by LLMs (e.g. in SQLite). To address this, we convert the output of “EXPLAIN" command into a human-readable text format that aligns more closely with the pretraining data of LLMs. The human-readable version of query plans consists of three key steps: (1) identifying and locating the relevant tables for the question, (2) performing operations such as counting, filtering, or matching between tables, and (3) delivering the final result by selecting the appropriate columns to return. This reasoning method complements the divide-and-conquer CoT strategy. While the divide-and-conquer approach is better suited for decomposing complex questions, the query plan approach excels when questions require more reasoning over the relationships between different parts of the question and the database schema. It systematically explains which tables to scan, how to match columns, and how to apply filters. Appendix Fig. 20 shows an example of a question that was answered correctly only by this method. Appendix Fig. 18 provides the prompt used for this reasoning strategy. ',
        'sub-headings': []}}]}},
   {'3.4 Query Fixer': {'content': ' In some cases, LLMs might generate queries that are syntactically incorrect. These queries are clear candidates for correction, as they fail to provide the correct answers. To address this, we apply an LLM-based query fixer that leverages the self-reflection (Shinn et al., 2024) method. The fixer reflects on the previously generated query, using feedback such as syntax error details or empty result sets to guide the correction process. We continue this iterative fixing approach up to a specified number of attempts, β (set to three in this paper). Appendix Fig. 21 demonstrates the prompt used for this query fixing step. ',
     'sub-headings': []}},
   {'3.5 Selection Agent': {'content': ' With three different methods for generating SQL queries, we can generate a set of candidate queries for any given question. The key challenge in this step is selecting the correct SQL query from this pool of candidates. A naive approach would be to measure consistency among the candidates by executing them, grouping them based on their execution results, and selecting a query from the largest group as the most likely correct answer. However, this would assume that the most consistent answer is always the best one, which is not always the case. Instead, we propose a more refined picking strategy, Algorithm 3, that relies on a selection agent. Given a set of candidates SQL queries C = {c1, c2, ..., cn}, the final responses are selected by finding 5 the candidate that has the highest score assigned by the selection model. This model θp can take k candidates and rank them based on how accurately each of them answers the given question. Concretely, we formulate the selection of the final response as: cf = arg max c∈C \uf8eb \uf8ec \uf8ed (n k) X i=1 θp(ci1, . . . , cik | Qu, Hu, D) \uf8f6 \uf8f7 \uf8f8, (1) where Qu refers to the user’s question, Hu is the provided hint, and D is the target database from which the question is being asked. In Eq. 1, we pass k candidates to the selection model to be ranked, with k being between 1 and n. In the extreme case of k = 1, the model is unable to make comparisons between candidates, which complicates the evaluation process for the model. As k increases, comparing more candidates makes the process more challenging for the model, as it needs to consider different aspects simultaneously. Consequently, we set k = 2 and train a model with a classification objective to compare only two candidates at a time. Having a set of high-quality and diverse candidates, the most straightforward solution is to employ off-the-shelf LLMs to make pairwise selections. However, experiments with Gemini-1.5-pro showed that using the LLM without fine-tuning resulted in only 58.01% binary classification accuracy. This is primarily due to the candidates being very similar to one another, requiring a fine-tuned model to learn the nuances and make more accurate decisions. To train the selection agent, we first generate candidate SQL queries on the training set (of Text-to-SQL benchmarks), and group them into clusters based on their execution results. For cases where at least one cluster contains correct queries and others contains incorrect ones, we create training examples in the form of tuples (Qu, Ci, Cj, Dij, yij), where Qu is the user’s question, Ci and Cj are the two candidate queries being compared, Dij is the database schema used by both candidates, and yij ∈0, 1 is the label indicating whether Ci or Cj is the correct query. To avoid order bias during training, we randomly shuffle the order of correct and incorrect queries in each pair. Since the number of cases with both correct and incorrect candidates is limited, for instances where no correct candidate exists, we include the ground truth SQL query in the prompt as a hint to guide the model in generating correct candidates. Algorithm 3 Picking the final SQL query from a pool of candidates. Input: Set of candidate SQL queries C = {c1, c2, ..., cn}, user question Qu, hint Hu, target database D, and a selection model θp, er(ci, D) as the execution result of ci on D 1: ri ←0 for all ci ∈C // Initialize the score ri for each candidate query to zero 2: for each distinct pair (ci, cj) where i ̸= j do 3: if er(ci, D) = er(cj, D) then 4: w ←i // ci is the winner if the execution results match 5: else 6: Si,j ←schema_union(ci, cj, D) // Construct union of schemas used in ci and cj 7: w ←θp(Si,j, Qu, Hu, ci, cj)w ∈{i, j} // Use binary classifier θp to select the winner, w ∈{i, j} 8: end if 9: rw ←rw + 1 // Increase the score of the winner cw by 1 10: end for 11: cf ←arg maxci∈C ri // Select the candidate with the highest score as the final SQL query cf 12: return cf In the pseudo-code for Algorithm 3, we begin by initializing a score of zero for each candidate query. Then, for every distinct pair of queries (ci, cj), we compare both (ci, cj) and (cj, ci) to mitigate any order bias, ensuring that both candidates in a pair are fairly evaluated. If both queries produce the same execution result on the database, we mark one as the winner and increment its score, as these results suggest consistency. If the execution results differ, we generate a union of the schema used by both queries and use the binary classifier to determine which query is more likely to be correct. The classifier takes into account the question, the two candidate queries, and the combined schema to make its decision. The winner’s score is then updated accordingly. After all comparisons, the candidate with the highest score is selected as the final query. In the rare case of a tie in the final scores, we break the tie by selecting one of the candidates arbitrarily. 6 ',
     'sub-headings': []}}]},
 '4 Online Synthetic Example Generation': {'content': ': Using M demonstrations for few-shot in-context learning has shown promising results on various related tasks (Pourreza & Rafiei, 2024a). Besides helping with specifying the task and illustrate the step-by-step process deriving the output, demonstrations constructed using relevant tables and columns can also help the model understand the underlying data schema. Based on this insight, we propose a synthetic demonstration generation strategy for Text-to-SQL – given the user question Qu, the target database D, and the selected columns ti (using a column selection approach similar to (Talaei et al., 2024)). Algorithm 2 Online Synthetic example generation strategy for Text-to-SQL. Input: User question Qu, additional user hint Hu, target database D and filtered relevant table columns t associated with the question, LLM θ, guidelines Rf for generating examples by SQL features, guidelines Rt for generating examples with filtered schema, and the numbers of examples to generate nf, nt respectively 1: P ←∅// {(qi, si) | qi, si ∈Σ∗}, where qi is input question , si is output SQL for the i-th example 2: P ←P ∪{θ(D, Rf, nf)} // Generate n examples with entire database by common SQL features 3: P ←P ∪{θ(t, Rt, nt)} // Generate examples with filtered columns to highlight correct schema usage 4: return P Algorithm 2 outlines the online synthetic example generation approach with two LLM generation steps. The first step focuses on generating illustrative examples with common SQL features described in the guideline Rf. The SQL features include equality and non-equality predicates, single table and multi-table JOIN, nested JOIN, ORDER BY and LIMIT, GROUP BY and HAVING, various aggregation functions. These are widely applicable SQL clauses and functions – the generated example SQL queries, incorporating these features, follow the BIRD SQL feature distribution (Appendix Fig 23a). The second step focuses on generating examples highlighting correct interpretation of the underlying data schema – the model θ is asked to generate examples using ti and that are similar to the examples outlined in Rt. Appendix A.10 provides the prompts used for the example generation). While a relevant example (e.g. showing a nested JOIN query with multiple tables) can be helpful for questions that require complex JOIN queries, it might also mislead the LLM for overuse (e.g. when a simple single table query is sufficient). This and the inherent ambiguity of natural language query qi, for which we draw the examples by relevance, make the example selection challenging. Thus, we generate and inject the examples to the prompt online per qi. We ask the LLM to generate many input-output pairs for in-context learning. The final set of synthetic examples for qi contains examples generated with both Rf and Rt. This ensures that the example set is diverse both in SQL features/clauses and the choice of relevant tables/columns used. The diversity of the example set is desirable to avoid over-fitting the output to certain patterns (e.g., the model always writes a SQL with JOIN if shown mostly JOIN examples). Mixing various examples for various SQL features and database tables with and without column filtering is observed to result in better generation quality overall (please see Appendix Table 8). ',
  'sub-headings': []},
 '4 Experiments': {'content': ' ',
  'sub-headings': [{'4.1 Datasets and Models': {'content': ' We evaluate the performance of the proposed CHASE-SQL framework on two widely-recognized cross-domain datasets: BIRD (Li et al., 2024c) and Spider (Yu et al., 2018). BIRD contains over 12,751 unique question-SQL pairs from 95 large databases, spanning more than 37 professional domains, with databases designed to resemble real-world scenarios, featuring messy data rows and complex schemas. Spider consists of 10,181 questions and 5,693 unique complex SQL queries across 200 databases, covering 138 domains. The Spider dataset is divided into non-overlapping training, development, and test sets similar to BIRD. For both, we use execution accuracy (EX), the official metric for their respective leaderboard, as the primary evaluation metric to compare methods. Details of the models and their hyperparameters are provided in Appendix section A.2. ',
     'sub-headings': []}},
   {'4.2 BIRD results': {'content': ' We present the end-to-end Text-to-SQL performance of the proposed CHASE-SQL framework using Claude- 3.5-sonnet and Gemini 1.5 pro on the BIRD development set, and Gemini 1.5 pro on the BIRD test set. We compare with both published methods (either with an available codebase and/or paper) and undisclosed methods. For a fair comparison with Gemini 1.5 pro, all LLM calls in the Claude-3.5-sonnet setting, except for the selection model, are made using Claude-3.5-sonnet (previously-trained selection model is reused). As shown in Table 2, CHASE-SQL with Gemini 1.5 pro achieves 73.01% accuracy on the BIRD development set and 73.0% on the BIRD holdout test set, outperforming all previous works and setting a new state-of-the-art performance. Table 2: Performance Comparison of different Text-to-SQL methods on BIRD benchmark. Method EX (Dev) EX (Test) Published CHASE-SQL + Gemini 1.5 (Ours) 73.01 73.0 CHASE-SQL + Claude 3.5 Sonnet (Ours) 69.53 – Distillery + GPT-4o (Maamari et al., 2024) 67.21 71.83 CHESS (Talaei et al., 2024) 65.00 66.69 MCS-SQL + GPT-4 (Lee et al., 2024) 63.36 65.45 SuperSQL (Li et al., 2024a) 58.5 62.66 Undisclosed Insights AI 72.16 70.26 AskData + GPT-4o 72.03 72.39 OpenSearch-v2 + GPT-4o 69.3 72.28 PURPLE-RED + GPT-4o 68.12 70.21 Arcwise + GPT-4o 67.99 66.21 ExSL + granite-34b-code 67.47 67.76 Table 3: Performance Comparison of different Text-to-SQL methods on Spider test set. Method EX Training with Spider MCS-SQL + GPT-4 (Lee et al., 2024) 89.6 ✓ CHASE-SQL + Gemini 1.5 (Ours) 87.6 ✗ CHESS (Talaei et al., 2024) 87.2 ✗ DAIL-SQL + GPT-4 (Gao et al., 2023) 86.6 ✓ DIN-SQL + GPT-4 (Pourreza & Rafiei, 2024a) 85.3 ✓ C3 + ChatGPT (Dong et al., 2023) 82.3 ✓ RESDSQL 3B (Li et al., 2023a) 79.9 ✓ DIN-SQL + CodeX (Pourreza & Rafiei, 2024a) 78.2 ✓ T5-3B+NatSQL (Rai et al., 2023) 78.0 ✓ Graphix-3B+PICARD (Li et al., 2023b) 77.6 ✓ ',
     'sub-headings': []}},
   {'4.3 Spider results': {'content': ' We assess the generalizability of the proposed CHASE-SQL by evaluating it in an end-to-end way on the Spider test set without modifying the few-shot samples in the prompts or training a new selection model, i.e. without using and data from the target distribution. This approach allows us to test the performance of CHASE-SQL on different unseen query and database distributions compared to the data from training distributions. Table 3 demonstrates that CHASE-SQL achieves an execution accuracy of 87.6% on the Spider test set, placing it second among methods that have undergone specific training or prompt optimization for the Spider dataset. This highlights the strong generalizability of CHASE-SQL and its potential for generating high quality Text-to-SQL for unseen samples coming from very different distributions and unique challenges. 7 ',
     'sub-headings': []}},
   {'4.4 Generator and selection performance': {'content': ' Table 4: Ablation studies on single candidate gen- eration performance of the candidate generators compared with original BIRD prompt + zero-shot CoT with Gemini 1.5 pro on the BIRD dev set. Selector is our final performance applying the se- lection agent to 21 candidates generated from all generators. Method Execution Accuracy (%) ∆(%) Baseline 57.75 - QP CoT 63.62 +5.87 DC CoT 63.92 +6.17 OS CoT 67.09 +9.34 Baseline w Query Fixer 61.58 +3.83 QP CoT w Query Fixer 65.51 +7.76 DC CoT w Query Fixer 65.77 +8.02 OS CoT w Query Fixer 68.02 +10.27 Generator + Fixer: To reveal performance of genera- tors, we conducted an ablation study to evaluate the per- formance of each candidate generation method before and after applying the query fixer. We compare the performance of the proposed generators in producing a single candidate query against the original BIRD prompt (Li et al., 2024c), augmented with zero-shot CoT reasoning (Kojima et al., 2022), which serves as the baseline for assessing the quality of prompts. The results, shown in Table 4, indicate that the proposed methods significantly improve SQL generation performance, compared to the naive baseline, towards the goal of producing high-quality candidates while maintain- ing diversity. Among the candidate generators, the online synthetic data generation approach produced an impressive performance of 68.02%, demonstrating its effectiveness in leveraging test-time compute to improve LLM performance by generating high-quality synthetic examples. Furthermore, the query fixer proved crucial, enhancing the quality of the candidate pool and increasing performance by nearly 2% across all candidate generators. Table 5: Evaluating the binary selection accuracy of the different selection models. ',
     'sub-headings': [{'Selection': {'content': ' Model Binary Acc. (%) Claude-3.5-sonnet 60.21 Gemini-1.5-pro 63.98 Tuned Gemma 2 9B 64.28 Tuned Gemini-1.5-flash 71.01 Selection: We conducted an analysis on the binary selection accu- racy of the selection agent for cases where, in a pairwise comparison, one candidate is correct and the other is incorrect. We exclude cases where both candidates are either correct or incorrect, as the selection would not affect the outcome since both candidates have the same label. We compare the performance of Claude-3.5-sonnet and Gemini-1.5-pro (both out-of-the-box without fine-tuning) with two fine-tuned models: 1) Gemma 2 9B and 2) Gemini-1.5-flash. As shown in Table 5, both fine-tuned models achieve higher accuracy than the untuned counterparts, demonstrating the importance of fine-tuning to teach the model about the specific preferences. ',
        'sub-headings': []}},
      {'Candidate Generation Analysis': {'content': ': We analyze the performance of each candidate generator method individually. To better understand the performance potential when effectively selecting the correct SQL query from the candidate pool, we generate seven candidate SQL queries from each generator method (21 candidates in total) for all samples in the BIRD development set. We determine this number of candidates based on the observation that increasing the candidate pool beyond 20 did not yield significant improvements, as illustrated in Fig. 2d. By assuming access to an oracle selection model that always selects the correct SQL query from the seven candidates, we calculate the upper-bound performance achievable for each generator. Conversely, by assuming an adversarial selection model that always selects the wrong SQL query, we determine the lower-bound performance. Fig. 2 illustrates the upper-bound and lower-bound performance for all three methods together with the performance of our selection agent. As shown, the upper-bound performance of the two different CoT methods is generally higher than that of the synthetic example generation method for different number of candidates. However, their lower-bound performance is also lower than the synthetic method. Lower-bound accuracy reflects cases where all candidates are correct, reducing the noise in the selection process since it doesn’t matter which candidate is chosen, so a higher lower-bound is preferred. This is evident in the selection agent’s performance, where a drop in the lower bound leads to diminishing returns from increasing the upper bound, causing the selection agent’s performance to plateau. Additionally, the upper-bound performance of combining all three methods reaches 82.79%, highlighting the significant room for improvement through better candidate picking methods. This demonstrates that the LLM’s parametric knowledge already contains the information needed to solve most questions, highlighting the need for ensemble approaches to effectively extract and utilize this knowledge. 8 (a) Upper-bound and lower-bound Accuracy for Divide and Conquer CoT (b) Upper-bound and lower-bound Accuracy for Online Synthetic Example (c) Upper-bound and lower-bound performance for Query Plan CoT. (d) Upper-bound performance of all three can- didate generators across different difficulty cat- egories. Figure 2: Comparison of the upper- and lower-bound performance of different candidate generators. Additionally, we evaluate the upper-bound performance by combining all candidates from three candidate generation methods across the simple, moderate, and challenging difficulty levels for the BIRD development set. These difficulty categories are assigned by human experts during the creation of the BIRD development set. Fig. 2d shows that, as expected, the upper-bound performance increases with the number of candidates across all difficulty levels. However, for the challenging and moderate classes, the improvement plateaus earlier than in the simple class, suggesting that generating more samples does not further improve the upper-bound performance for these two difficulty levels. Fig. 2 presents a Venn diagram showcasing the performance of three generation methods: Query Plan, Divide and Conquer, and with Synthetic Examples. The numbers within the intersecting regions represent the instances where multiple methods generated at least one correct candidate. This diagram visually highlights the unique contributions of each method, which indicates the necessity of using all three generators. Additionally, in Fig. 3b, we compare the number of correct queries generated by each SQL generation method that are not correct by the other generators. The divide-and-conquer approach outperforms the others on challenging questions, while the query plan method excels on moderately difficult queries. To further analyze the performance of the generators across different domains and varying numbers of columns and tables, we compare the number of correct queries generated for each database, as shown in Appendix Fig. 4. As illustrated, both CoT methods generally perform similarly across databases, while the online synthetic example generation method significantly increases diversity, resulting in more correct answers overall across different databases. ',
        'sub-headings': []}},
      {'Selection Agent Analysis': {'content': ': We evaluate the query-picking performance by comparing the Text-to-SQL execution accuracy of the selection agent with the self-consistency method (using majority voting) Wang et al. (2022), an oracle model (upper bound), and an adversarial model (lower bound). To conduct the evaluation, we generate 10 samples from each candidate generation method using two different sampling temperatures: 0.5 and 1.8. The results, shown in Table 6, demonstrate that the selection agent significantly outperforms the self-consistency method with a large margin, roughly 6%. As expected, increasing the sampling temperature 9 35 30 33 38 72 23 1045 ',
        'sub-headings': []}},
      {'Query Plan': {'content': ' ', 'sub-headings': []}},
      {'Synthetic Example': {'content': ' ', 'sub-headings': []}},
      {'Divide and Conquer': {'content': ' Unsolved Questions: 258 (a) Venn diagram illustrating the num- ber of instances for which each method: Query Plan, Synthetic Example, Divide and Conquer, produces at least one cor- rect candidate. The overlap regions repre- sent multiple methods generating correct candidates. (b) Number of correct queries across different complexity levels that were answered by each method. Figure 3: Comparison of SQL generation methods: Venn diagram showing unique and overlapping correct answers (left) and the performance across different complexity levels (right). raises the upper bound but also lowers the lower bound. This effect is more pronounced for the synthetic data generation method compared to the two CoT methods, mainly because LLMs generate reasoning steps before producing the final SQL query, which helps mitigate the randomness introduced by high-temperature sampling. The performance with self-consistency method generally decreases as temperature increases, since the majority cluster becomes smaller with more random queries. However, the proposed trained selection agent is less affected by temperature scaling and, in two cases, even improved its performance with a more diverse pool of samples. Table 6: Performance comparison of different picking methods on the candidates generated by the candidate generators on BIRD development set with two different temperatures. QP refers to query plan COT, DC refers to divide and conquer COT, and OS is the online synthetic example generation method. Picking Method QP (T=0.5) QP (T=1.8) DC (T=0.5) DC (T=1.8) OS (T=0.5) OS (T=1.8) Lower Bound 50.46 48.63 51.37 47.39 60.43 50.98 Upper Bound 78.55 80.44 78.42 79.34 74.77 79.66 Self-consistency 65.78 65.51 66.43 64.41 67.34 66.88 Our Selection Agent 71.7 71.73 71.31 70.53 70.4 71.38 ',
        'sub-headings': []}}]}},
   {'4.5 Ablation Studies': {'content': ' In the previous sections, we evaluate the importance of the selection agent and each candidate generation method. Next, we focus on the analysis of the remaining components of CHASE-SQL: LSH for value retrieval, the query fixer, and three reasoning strategies (QP, OS, and DC). Table 7 shows the performance of CHASE- SQL without each of these steps, highlighting their significance in achieving higher-quality performance. The results demonstrate the contribution of each component, where removing LSH, the query fixer, or any of the candidate generators leads to a reduction in execution accuracy, further validating the importance of these components of CHASE-SQL. Moreover, the table compares the performance of our binary selection agent with two other selection methods: self-consistency (Wang et al., 2022) and a ranker agent. The ranker agent receives all candidates generated by our three candidate generators in a single prompt, compares them, and produce a ranking for each. For the ranker agent, we select the query with the lowest rank as the best answer. The binary selection agent significantly outperforms both the self-consistency and ranker agents, demonstrating the effectiveness of the proposed method. 10 ',
     'sub-headings': []}}]},
 '5 Conclusion': {'content': ' Table 7: Ablation studies on the performance of CHASE- SQL after removing the query fixer, LSH for value re- trieval, and reasoning strategies, i.e., QP, OS, and DC. Method Execution Accuracy (%) ∆(%) CHASE-SQL All 73.01 - CHASE-SQL w self-consistency 68.84 -4.17 CHASE-SQL w ranker agent 65.51 -7.5 CHASE-SQL w/o LSH 70.09 -2.92 CHASE-SQL w/o Query Fixer 69.23 -3.78 CHASE-SQL w/o QP 72.36 -0.65 CHASE-SQL w/o OS 72.16 -0.85 CHASE-SQL w/o DC 71.77 -1.24 We introduce a novel agentic framework, CHASE-SQL, to leverage test-time compute for generating diverse, high-quality SQL queries and accurately selecting the correct one. We propose multiple chain-of-thought prompting methods and an online synthetic example generation technique, along with a query selection mechanism that scores candidates based on pairwise comparisons. Our framework, CHASE-SQL, sets a new state-of-the-art in the notable public Text-to-SQL leaderboard (at the time of the submission), demon- strating the effectiveness of test-time computation for both generating diverse queries and selecting the most accurate response. CHASE-SQL addresses key issues like query diversity and selection optimization, paving the way for further improvements in complex reasoning tasks encountered at real-world Text-to-SQL challenges. ',
  'sub-headings': []},
 'Acknowledgments': {'content': ' We would like to thank Per Jacobsson, Raj Sinha, Zeke Miller, Reza Sherkat, James Su, Zhixian Yan, David Culler, and Xiance Si, for their valuable comments and feedbacks on our paper. We would also like to thank the BIRD team for their invaluable assistance with the evaluation of the BIRD test set. ',
  'sub-headings': []},
 'References': {'content': ' Ion Androutsopoulos, Graeme D Ritchie, and Peter Thanisch. Natural language interfaces to databases–an introduction. Natural language engineering, 1(1):29–81, 1995. Ruichu Cai, Jinjie Yuan, Boyan Xu, and Zhifeng Hao. Sadga: Structure-aware dual graph aggregation network for text-to-sql. Advances in Neural Information Processing Systems, 34:7664–7676, 2021. Ruisheng Cao, Lu Chen, Zhi Chen, Yanbin Zhao, Su Zhu, and Kai Yu. Lgesql: line graph enhanced text-to-sql model with mixed local and non-local relations. arXiv preprint arXiv:2106.01093, 2021. Mark Chen, Jerry Tworek, Heewoo Jun, Qiming Yuan, Henrique Ponde de Oliveira Pinto, Jared Kaplan, Harri Edwards, Yuri Burda, Nicholas Joseph, Greg Brockman, et al. Evaluating large language models trained on code. arXiv preprint arXiv:2107.03374, 2021. Xinyun Chen, Maxwell Lin, Nathanael Schärli, and Denny Zhou. Teaching large language models to self-debug. arXiv preprint arXiv:2304.05128, 2023. DongHyun Choi, Myeong Cheol Shin, EungGyun Kim, and Dong Ryeol Shin. Ryansql: Recursively applying sketch-based slot fillings for complex text-to-sql in cross-domain databases. Computational Linguistics, 47 (2):309–332, 2021. Mayur Datar, Nicole Immorlica, Piotr Indyk, and Vahab S Mirrokni. Locality-sensitive hashing scheme based on p-stable distributions. In Proceedings of the twentieth annual symposium on Computational geometry, pp. 253–262, 2004. Xuemei Dong, Chao Zhang, Yuhang Ge, Yuren Mao, Yunjun Gao, Jinshu Lin, Dongfang Lou, et al. C3: Zero-shot text-to-sql with chatgpt. arXiv preprint arXiv:2307.07306, 2023. Dawei Gao, Haibin Wang, Yaliang Li, Xiuyu Sun, Yichen Qian, Bolin Ding, and Jingren Zhou. Text-to-sql empowered by large language models: A benchmark evaluation. arXiv preprint arXiv:2308.15363, 2023. Jonathan Herzig, Paweł Krzysztof Nowak, Thomas Müller, Francesco Piccinno, and Julian Martin Eisenschlos. Tapas: Weakly supervised table parsing via pre-training. arXiv preprint arXiv:2004.02349, 2020. 11 Wonseok Hwang, Jinyeong Yim, Seunghyun Park, and Minjoon Seo. A comprehensive exploration on wikisql with table-aware word contextualization. arXiv preprint arXiv:1902.01069, 2019. George Katsogiannis-Meimarakis and Georgia Koutrika. A survey on deep learning approaches for text-to-sql. The VLDB Journal, 32(4):905–936, 2023. Takeshi Kojima, Shixiang Shane Gu, Machel Reid, Yutaka Matsuo, and Yusuke Iwasawa. Large language models are zero-shot reasoners. Advances in neural information processing systems, 35:22199–22213, 2022. Dongjun Lee, Choongwon Park, Jaehyuk Kim, and Heesoo Park. Mcs-sql: Leveraging multiple prompts and multiple-choice selection for text-to-sql generation. arXiv preprint arXiv:2405.07467, 2024. Boyan Li, Yuyu Luo, Chengliang Chai, Guoliang Li, and Nan Tang. The dawn of natural language to sql: Are we fully ready? arXiv preprint arXiv:2406.01265, 2024a. Fei Li and Hosagrahar V Jagadish. Constructing an interactive natural language interface for relational databases. Proceedings of the VLDB Endowment, 8(1):73–84, 2014. Haoyang Li, Jing Zhang, Cuiping Li, and Hong Chen. Resdsql: Decoupling schema linking and skeleton parsing for text-to-sql. In Proceedings of the AAAI Conference on Artificial Intelligence, volume 37, pp. 13067–13075, 2023a. Haoyang Li, Jing Zhang, Hanbing Liu, Ju Fan, Xiaokang Zhang, Jun Zhu, Renjie Wei, Hongyan Pan, Cuiping Li, and Hong Chen. Codes: Towards building open-source language models for text-to-sql. Proceedings of the ACM on Management of Data, 2(3):1–28, 2024b. Jinyang Li, Binyuan Hui, Reynold Cheng, Bowen Qin, Chenhao Ma, Nan Huo, Fei Huang, Wenyu Du, Luo Si, and Yongbin Li. Graphix-t5: Mixing pre-trained transformers with graph-aware layers for text-to-sql parsing. arXiv preprint arXiv:2301.07507, 2023b. Jinyang Li, Binyuan Hui, Ge Qu, Jiaxi Yang, Binhua Li, Bowen Li, Bailin Wang, Bowen Qin, Ruiying Geng, Nan Huo, et al. Can llm already serve as a database interface? a big bench for large-scale database grounded text-to-sqls. Advances in Neural Information Processing Systems, 36, 2024c. Yujia Li, David Choi, Junyoung Chung, Nate Kushman, Julian Schrittwieser, Rémi Leblond, Tom Eccles, James Keeling, Felix Gimeno, Agustin Dal Lago, et al. Competition-level code generation with alphacode. Science, 378(6624):1092–1097, 2022. Karime Maamari, Fadhil Abubaker, Daniel Jaroslawicz, and Amine Mhedhbi. The death of schema linking? text-to-sql in the age of well-reasoned language models. arXiv preprint arXiv:2408.07702, 2024. Ansong Ni, Srini Iyer, Dragomir Radev, Veselin Stoyanov, Wen-tau Yih, Sida Wang, and Xi Victoria Lin. Lever: Learning to verify language-to-code generation with execution. In International Conference on Machine Learning, pp. 26106–26128. PMLR, 2023. Rubén Pérez-Mercado, Antonio Balderas, Andrés Muñoz, Juan Francisco Cabrera, Manuel Palomo-Duarte, and Juan Manuel Dodero. Chatbotsql: Conversational agent to support relational database query language learning. SoftwareX, 22:101346, 2023. Mohammadreza Pourreza and Davood Rafiei. Din-sql: Decomposed in-context learning of text-to-sql with self-correction. Advances in Neural Information Processing Systems, 36, 2024a. Mohammadreza Pourreza and Davood Rafiei. Dts-sql: Decomposed text-to-sql with small large language models. arXiv preprint arXiv:2402.01117, 2024b. Mohammadreza Pourreza, Ruoxi Sun, Hailong Li, Lesly Miculicich, Tomas Pfister, and Sercan O Arik. Sql-gen: Bridging the dialect gap for text-to-sql via synthetic data and model merging. arXiv preprint arXiv:2408.12733, 2024. 12 Abdul Quamar, Vasilis Efthymiou, Chuan Lei, and Fatma Özcan. Natural language interfaces to data. Found. Trends Databases, 11(4):319–414, 2022. doi: 10.1561/1900000078. URL https://doi.org/10. 1561/1900000078.',
  'sub-headings': []}}

In [4]:
# def extract_all_content(json_data):
#     contents = []

#     def recursive_extract(data):
#         if isinstance(data, dict):
#             for key, value in data.items():
#                 if key == "content" and isinstance(value, str):
#                     contents.append(value)
#                 else:
#                     recursive_extract(value)
#         elif isinstance(data, list):
#             for item in data:
#                 recursive_extract(item)

#     recursive_extract(json_data)
#     return " ".join(contents)

# extract_all_content(chunks['1 Introduction'])

In [4]:
from t2j.extractor import FieldExtractor
from t2j.prompts import Prompts
from t2j.prem_sdk import PremSDK

promptsClass = Prompts()
model = PremSDK()

e = FieldExtractor(model, promptsClass)
e.extract(chunks, res)

{'path': 'related_work[]', 'type': 'array<string>', 'description': 'List of strings, with related works done; keep the points short and precise', 'node_type': 'trunk'}
{'path': 'approach[]', 'type': 'array<object>', 'description': 'List of methods used in the approach', 'node_type': 'trunk'}
{'output': ['3 Methods -> sub-headings[0] -> 3.1 Overall Framework', '3 Methods -> sub-headings[1] -> 3.2 Value Retrieval', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[0] -> Divide and Conquer CoT', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[1] -> Query Plan CoT', '3 Methods -> sub-headings[3] -> 3.4 Query Fixer', '3 Methods -> sub-headings[4] -> 3.5 Selection Agent']}
 This section outlines the proposed CHASE-SQL framework, which consists of four primary components: 1) Value retrieval, 2) Candidate generator, 3) Query fixer, and 4) Selection agen

[{'schema_field': {'path': 'approach[]',
   'type': 'array<object>',
   'description': 'List of methods used in the approach',
   'node_type': 'trunk'},
  'data': [{'approach': [{'approach_name': 'Value retrieval',
      'description': 'Retrieving relevant database values for SQL query generation.',
      'steps': [{'step_title': 'Keyword Extraction',
        'details': 'Extract keywords from the given question using an LLM prompted with few-shot examples.'},
       {'step_title': 'LSH Retrieval',
        'details': 'Use locality-sensitive hashing (LSH) to retrieve the most syntactically-similar words.'},
       {'step_title': 'Re-ranking',
        'details': 'Re-rank retrieved words based on embedding-based similarity and edit distance.'}],
      'improvements': {'metric': 'Robustness',
       'value_added': 'Improves robustness to typos and considers keyword semantics.'}},
     {'approach_name': 'Candidate generator',
      'description': 'Generate diverse candidate SQL queries using

In [6]:
{'output': [{'approach': [{'approach_name': 'Value retrieval', 'description': 'Retrieving relevant database values for use in SQL clauses.', 'steps': [{'step_title': 'Keyword extraction', 'details': 'Extract keywords from the given question using an LLM prompted with few-shot examples.'}, {'step_title': 'LSH retrieval', 'details': 'Use locality-sensitive hashing to retrieve the most syntactically-similar words.'}, {'step_title': 'Re-ranking', 'details': 'Re-rank retrieved words based on embedding-based similarity and edit distance.'}], 'improvements': {'metric': 'Robustness', 'value_added': 'Improved handling of typos and keyword semantics during retrieval.'}}, {'approach_name': 'Candidate generator', 'description': 'Generate diverse candidate queries using LLMs.', 'steps': [{'step_title': 'Diversity in generation', 'details': 'Increase the next token sampling temperature and shuffle the order of columns and tables in the prompt.'}], 'improvements': {'metric': 'Diversity', 'value_added': 'Increased likelihood of generating at least one correct answer.'}}, {'approach_name': 'Query fixer', 'description': 'Fix syntactically incorrect queries using self-reflection.', 'steps': [{'step_title': 'Iterative fixing', 'details': 'Reflect on the previously generated query using feedback to guide the correction process.'}], 'improvements': {'metric': 'Accuracy', 'value_added': 'Corrects queries that fail to provide the correct answers.'}}, {'approach_name': 'Selection agent', 'description': 'Select the final response using a trained selection agent.', 'steps': [{'step_title': 'Pairwise comparison', 'details': 'Compare candidates pairwise using a selection model to pick the correct answer.'}], 'improvements': {'metric': 'Selection accuracy', 'value_added': 'More accurate decisions by learning the nuances between similar candidates.'}}]}]}

{'output': [{'approach': [{'approach_name': 'Value retrieval',
     'description': 'Retrieving relevant database values for use in SQL clauses.',
     'steps': [{'step_title': 'Keyword extraction',
       'details': 'Extract keywords from the given question using an LLM prompted with few-shot examples.'},
      {'step_title': 'LSH retrieval',
       'details': 'Use locality-sensitive hashing to retrieve the most syntactically-similar words.'},
      {'step_title': 'Re-ranking',
       'details': 'Re-rank retrieved words based on embedding-based similarity and edit distance.'}],
     'improvements': {'metric': 'Robustness',
      'value_added': 'Improved handling of typos and keyword semantics during retrieval.'}},
    {'approach_name': 'Candidate generator',
     'description': 'Generate diverse candidate queries using LLMs.',
     'steps': [{'step_title': 'Diversity in generation',
       'details': 'Increase the next token sampling temperature and shuffle the order of columns and tab

In [5]:
['3 Methods -> sub-headings[0] -> 3.1 Overall Framework', '3 Methods -> sub-headings[1] -> 3.2 Value Retrieval', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[0] -> Divide and Conquer CoT', '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[1] -> Query Plan CoT', '3 Methods -> sub-headings[3] -> 3.4 Query Fixer', '3 Methods -> sub-headings[4] -> 3.5 Selection Agent']

['3 Methods -> sub-headings[0] -> 3.1 Overall Framework',
 '3 Methods -> sub-headings[1] -> 3.2 Value Retrieval',
 '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation',
 '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[0] -> Divide and Conquer CoT',
 '3 Methods -> sub-headings[2] -> 3.3 Multi-path Candidate Generation -> sub-headings[1] -> Query Plan CoT',
 '3 Methods -> sub-headings[3] -> 3.4 Query Fixer',
 '3 Methods -> sub-headings[4] -> 3.5 Selection Agent']

In [7]:
field_path = "approach[]"
res


[{'path': 'title',
  'type': 'string',
  'description': 'Title of the paper or document',
  'node_type': 'trunk'},
 {'path': 'related_work[]',
  'type': 'array<string>',
  'description': 'List of strings, with related works done; keep the points short and precise',
  'node_type': 'trunk'},
 {'path': 'authors[]',
  'type': 'array<string>',
  'description': 'List of authors of the paper',
  'node_type': 'trunk'},
 {'path': 'abstract',
  'type': 'string',
  'description': 'Up to 5 bullet points describing what is new in the approach',
  'node_type': 'trunk'},
 {'path': 'approach[]',
  'type': 'array<object>',
  'description': 'List of methods used in the approach',
  'node_type': 'trunk'},
 {'path': 'approach[].approach_name',
  'type': 'string',
  'description': 'Name of the method or technique',
  'node_type': 'branch'},
 {'path': 'approach[].description',
  'type': 'string',
  'description': 'Brief summary of what the method is',
  'node_type': 'branch'},
 {'path': 'approach[].steps[]'

In [8]:
result = [r for r in res if r['path'].startswith(field_path)]

In [9]:
result

[{'path': 'approach[]',
  'type': 'array<object>',
  'description': 'List of methods used in the approach',
  'node_type': 'trunk'},
 {'path': 'approach[].approach_name',
  'type': 'string',
  'description': 'Name of the method or technique',
  'node_type': 'branch'},
 {'path': 'approach[].description',
  'type': 'string',
  'description': 'Brief summary of what the method is',
  'node_type': 'branch'},
 {'path': 'approach[].steps[]',
  'type': 'array<object>',
  'description': 'Step-by-step breakdown of the approach',
  'node_type': 'branch'},
 {'path': 'approach[].steps[].step_title',
  'type': 'string',
  'description': 'title of the step',
  'node_type': 'branch'},
 {'path': 'approach[].steps[].details',
  'type': 'string',
  'description': 'more details regarding the step',
  'node_type': 'branch'},
 {'path': 'approach[].improvements',
  'type': 'object',
  'description': 'Impact of the approach',
  'node_type': 'branch'},
 {'path': 'approach[].improvements.metric',
  'type': 'str

In [13]:
def create_empty_json_from_skeleton(skeleton):
    result = {}
    
    for item in skeleton:
        path = item['path']
        parts = path.split('.')
        current = result
        
        for i, part in enumerate(parts):
            if part.endswith('[]'):
                # Handle array case
                key = part[:-2]
                if key not in current:
                    current[key] = []
                    # For arrays of objects, add an empty object if it's the last part
                    if i == len(parts) - 1 and item['type'] == 'array<object>':
                        current[key].append({})
                # Move into the array (or create object if needed)
                if i < len(parts) - 1:
                    if not current[key]:
                        current[key].append({})
                    current = current[key][0]
                else:
                    # For leaf arrays that aren't objects, just leave as empty array
                    pass
            else:
                # Handle object case
                if part not in current:
                    if i == len(parts) - 1:
                        # Leaf node - set appropriate empty value based on type
                        if item['type'] == 'string':
                            current[part] = ""
                        elif item['type'] == 'object':
                            current[part] = {}
                        elif item['type'].startswith('array'):
                            current[part] = []
                    else:
                        current[part] = {}
                current = current[part]
    
    return result

# Given skeleton
skeleton = [
    {'path': 'approach[]', 'type': 'array<object>', 'description': 'List of methods used in the approach', 'node_type': 'trunk'},
    {'path': 'approach[].approach_name', 'type': 'string', 'description': 'Name of the method or technique', 'node_type': 'branch'},
    {'path': 'approach[].description', 'type': 'string', 'description': 'Brief summary of what the method is', 'node_type': 'branch'},
    {'path': 'approach[].steps[]', 'type': 'array<object>', 'description': 'Step-by-step breakdown of the approach', 'node_type': 'branch'},
    {'path': 'approach[].steps[].step_title', 'type': 'string', 'description': 'title of the step', 'node_type': 'branch'},
    {'path': 'approach[].steps[].details', 'type': 'string', 'description': 'more details regarding the step', 'node_type': 'branch'},
    {'path': 'approach[].improvements', 'type': 'object', 'description': 'Impact of the approach', 'node_type': 'branch'},
    {'path': 'approach[].improvements.metric', 'type': 'string', 'description': '', 'node_type': 'branch'},
    {'path': 'approach[].improvements.value_added', 'type': 'string', 'description': '', 'node_type': 'branch'}
]

# Create the empty JSON
empty_json = create_empty_json_from_skeleton(skeleton)

# Print the result
import json
print(json.dumps(empty_json, indent=2))

{
  "approach": [
    {
      "approach_name": "",
      "description": "",
      "steps": [
        {
          "step_title": "",
          "details": ""
        }
      ],
      "improvements": {
        "metric": "",
        "value_added": ""
      }
    }
  ]
}


In [ ]:
output = model.generate(prompt)

In [ ]:
model.extract_json(output)

{'output': ['2 Related Work']}

In [ ]:
data[model.extract_json(output)['output'][0]]

{'content': 2, 'sub-headings': []}

In [ ]:
def replace_all_content_with_ids(data):
    counter = {"value": 1}
    mapping = {}

    def recurse(obj):
        if isinstance(obj, dict):
            for key, value in obj.items():
                if key == "content":
                    mapping[counter["value"]] = value
                    obj[key] = counter["value"]
                    counter["value"] += 1
                else:
                    recurse(value)

            if "sub-headings" in obj and isinstance(obj["sub-headings"], list):
                for item in obj["sub-headings"]:
                    for sub_key, sub_value in item.items():
                        recurse(sub_value)

        elif isinstance(obj, list):
            for item in obj:
                recurse(item)

    recurse(data)
    return data, mapping

In [ ]:
import copy

copy_chunks = copy.deepcopy(chunks)
data, mapping = replace_all_content_with_ids(copy_chunks)

In [ ]:
data

{'1 Introduction': {'content': 1, 'sub-headings': []},
 '2 Related Work': {'content': 2, 'sub-headings': []},
 '3 Methods': {'content': 3,
  'sub-headings': [{'3.1 Overall Framework': {'content': 13,
     'sub-headings': []}},
   {'3.2 Value Retrieval': {'content': 14, 'sub-headings': []}},
   {'3.3 Multi-path Candidate Generation': {'content': 15,
     'sub-headings': [{'Divide and Conquer CoT': {'content': 18,
        'sub-headings': []}},
      {'Query Plan CoT': {'content': 19, 'sub-headings': []}}]}},
   {'3.4 Query Fixer': {'content': 20, 'sub-headings': []}},
   {'3.5 Selection Agent': {'content': 21, 'sub-headings': []}}]},
 '4 Online Synthetic Example Generation': {'content': 22, 'sub-headings': []},
 '4 Experiments': {'content': 23,
  'sub-headings': [{'4.1 Datasets and Models': {'content': 41,
     'sub-headings': []}},
   {'4.2 BIRD results': {'content': 42, 'sub-headings': []}},
   {'4.3 Spider results': {'content': 43, 'sub-headings': []}},
   {'4.4 Generator and selectio

In [15]:
mapping[2]

' Early Text-to-SQL methods predominantly utilized sequence-to-sequence architectures, encoding user queries and database schemas using models such as Graph Neural Networks (GNNs), Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTM) networks, and pre-trained transformer encoders (Cai et al., 2021; Cao et al., 2021; Hwang et al., 2019). On the decoding side, these systems employed either slot-filling or auto-regressive modelling approaches to construct the final SQL queries from the encoded inputs (Choi et al., 2021; Wang et al., 2019). Additionally, tabular language models like TaBERT (Yin et al., 2020), 2 TaPas (Herzig et al., 2020), and Grappa (Yu et al., 2020) have been developed to encode both tables and textual data effectively. However, the landscape has evolved with the widespread use of LLMs, which have largely replaced earlier methods with their superior performance (Katsogiannis-Meimarakis & Koutrika, 2023; Quamar et al., 2022). Initially, efforts concentrated on 